<a href="https://colab.research.google.com/github/JaimeSCotta/IAProject/blob/main/notebook/gastronomicSystemNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Valoración del sistema gastronómico NLP

El objetivo principal de este fichero consiste en diseñar, implementar y evaluar un modelo que resuelva la tarea de extraer la valoración del sistema gastronómico de los países Europeos y analizar si dicho comentario es positivo o negativo (análisis de sentimientos).

Para el desarrollo completo, se han identificado tres tareas principales:

- Preprocesamiento y preparación de los datos.

- Diseño, implementación y pruebas de modelos de lenguaje.

- Conclusiones y evaluación del modelo final.

Cada una de estas tres tareas principales se encuentra recogida en una sección específica del presente notebook.

En la primera sección, **Datos**, se recoge la obtención, tratamiento y modificación de los datos para dejarlos preparados para su utilización.

En la segunda sección, **Modelos**, se recogen las distintas aproximaciones que se han planteado para resolver la tarea planteada. Esta sección contiene todos los modelos definidos con su correspondiente entrenamiento y pruebas.

En la tercera sección, **Conclusiones**, se recoge la evaluación del modelo final y las conclusiones alcanzadas.

#1. Datos

En esta sección se aborda la obtención, análisis, tratamiento y preparación de los datos de entrenamiento para su posterior uso. Además, se obtienen los datos de validación. El dataset empleado en esta tarea ha sido obtendio de *Kaggel* (https://www.kaggle.com/datasets/damienbeneschi/krakow-ta-restaurans-data-raw).

Las tres subsecciones de este apartado se corresponden con los 3 tratamientos distintos del conjunto de entrenamiento que se han planteado.

- **Datos originales**: Se corresponde con el tratamiento sobre los datos originales sin modificar los textos (`Reviews`). Este tratamiento se centra más en preparar los datos tal y como vienen para poder usarlos y menos en investigar nuevas formas de representarlos y sacarles partido.
- **Datos divididos**: Se corresponde con un tratamiento alternativo extra en el que, en vez de limitarnos a usar las `Reviews` que nos vienen ya dadas, se desempareja cada par de reseñas para contar con un único conjunto de datos con todos las `Reviews` separadas. Esta aproximación permite aumentar el conjunto de datos proporcionado. Esto ayuda a un mejor entrenamiento y aprendizaje del modelo.

- **Datos siamese**: Se corresponde con un tratamiento diferente en el que, se desempareja cada par de reseñas para contar con un conjunto de datos donde se tienen ambas reviews en diferentes columnas, esto es, para posteriormente realizar el embbeding por separado y concatenar el resultado. Esta aproximación se realiza para obtener otro conjunto de resultados que comparar con los previos.


Cabe destacar que, dentro de la subsección de **Datos Originales**, también se encuentra la obtención y análisis de los datos con los que se evaluará el modelo final.

##1.1 Datos originales

###1.1.1 Imports

In [ ]:
import numpy as np
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###1.1.2 Carga de los datos

Inicialmente, se leen los datos en crudo con los que se va a trabajar y se almacenan en un DataFrame llamado `datos`. Como se puede observar, el DataFrame tiene un total de 125527 registros, es decir, valoraciones entre restaurantes, y consta de 11 columnas:
- `RestaurantID: int64` -> identificador del restaurante (de 31 ciudades Europeas).
- `Name: object` -> nombre del restaurante en cuestión.
- `City: object` -> ubicación de la ciudad del restaurante.
- `Cuisine Style: list(object)` -> estilo(s) de cocina del restaurante.
- `Ranking: float64` -> clasificación del restaurante entre el número total de restaurantes en la ciudad (medida similar al ID).
- `Rating: float64` -> puntuación del restaurante en una escala del 1 al 5.
- `Price Range: object` -> rango de precios del restaurante dividido en 3 categorías (\$, \$$ o bien \$$$).
- `Number of Reviews: float64` -> numero total de reseñas que ha recibido el restaurante.
- `Reviews: object` -> contenido de las dos reseñas principales del restaurante junto con la fecha de publicación.
- `URL_TA: object` -> parte de la URL de la página detallada del restaurante que sigue a 'www.tripadvisor.com'.
- `ID_TA: object` -> identificación del restaurante en la base de datos de TA obtenida con una letra y un número.


In [ ]:
datos = pd.read_csv("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/TA_restaurants_curated.csv")
datos.head(3)

,Unnamed: 0,Name,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,0,Martine of Martine's Table,Amsterdam,"['French', 'Dutch', 'European']",1.0,5.0,$$ - $$$,136.0,"[['Just like home', 'A Warm Welcome to Wintry ...",/Restaurant_Review-g188590-d11752080-Reviews-M...,d11752080
1,1,De Silveren Spiegel,Amsterdam,"['Dutch', 'European', 'Vegetarian Friendly', '...",2.0,4.5,$$$$,812.0,"[['Great food and staff', 'just perfect'], ['0...",/Restaurant_Review-g188590-d693419-Reviews-De_...,d693419
2,2,La Rive,Amsterdam,"['Mediterranean', 'French', 'International', '...",3.0,4.5,$$$$,567.0,"[['Satisfaction', 'Delicious old school restau...",/Restaurant_Review-g188590-d696959-Reviews-La_...,d696959


In [ ]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125527 entries, 0 to 125526
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         125527 non-null  int64  
 1   Name               125527 non-null  object 
 2   City               125527 non-null  object 
 3   Cuisine Style      94176 non-null   object 
 4   Ranking            115876 non-null  float64
 5   Rating             115897 non-null  float64
 6   Price Range        77672 non-null   object 
 7   Number of Reviews  108183 non-null  float64
 8   Reviews            115911 non-null  object 
 9   URL_TA             125527 non-null  object 
 10  ID_TA              125527 non-null  object 
dtypes: float64(3), int64(1), object(7)
memory usage: 10.5+ MB


###1.1.3 Preprocesamiento de los datos

En la tarea que se presenta, es necesario realizar un preprocesamiento de los datos, por ello en las siguientes celdas de código se trataran los datos obtenidos del dataset proporcionado. Se eliminarán datos nulos o vacíos, así como definir el umbral necesario para la correcta clasificación dada la tarea *sentiment analysis*.

In [ ]:
type(datos.Reviews)

pandas.core.series.Series

In [ ]:
datos.Reviews[1]

"[['Great food and staff', 'just perfect'], ['01/06/2018', '01/04/2018']]"

In [ ]:
{att : datos[datos[att].isnull()].shape[0] for att in datos.columns}

{'Unnamed: 0': 0,
 'Name': 0,
 'City': 0,
 'Cuisine Style': 31351,
 'Ranking': 9651,
 'Rating': 9630,
 'Price Range': 47855,
 'Number of Reviews': 17344,
 'Reviews': 9616,
 'URL_TA': 0,
 'ID_TA': 0}

In [ ]:
datos.dropna(subset = ['Rating','Reviews'] , inplace=True)

In [ ]:
{att : datos[datos[att].isnull()].shape[0] for att in datos.columns}

{'Unnamed: 0': 0,
 'Name': 0,
 'City': 0,
 'Cuisine Style': 26845,
 'Ranking': 145,
 'Rating': 0,
 'Price Range': 40418,
 'Number of Reviews': 7710,
 'Reviews': 0,
 'URL_TA': 0,
 'ID_TA': 0}

Es por ello, que se **eliminaran las columnas** que no son necesarias. Para el problema planteado se requiere unicamente la columna `Reviews` y la columna `Rating`.

In [ ]:
datos.columns

Index(['Unnamed: 0', 'Name', 'City', 'Cuisine Style', 'Ranking', 'Rating',
       'Price Range', 'Number of Reviews', 'Reviews', 'URL_TA', 'ID_TA'],
      dtype='object')

In [ ]:
col_validas = ['Reviews', 'Rating']

In [ ]:
col_eliminar = [columna for columna in datos.columns if columna not in col_validas]

In [ ]:
col_eliminar

['Unnamed: 0',
 'Name',
 'City',
 'Cuisine Style',
 'Ranking',
 'Price Range',
 'Number of Reviews',
 'URL_TA',
 'ID_TA']

In [ ]:
datos = datos.drop(col_eliminar, axis=1)

In [ ]:
datos

,Rating,Reviews
0,5.0,"[['Just like home', 'A Warm Welcome to Wintry ..."
1,4.5,"[['Great food and staff', 'just perfect'], ['0..."
2,4.5,"[['Satisfaction', 'Delicious old school restau..."
3,5.0,"[['True five star dinner', 'A superb evening o..."
4,4.5,"[['Best meal.... EVER', 'super food experience..."
...,...,...
125450,1.0,"[[], []]"
125451,1.0,"[['Poor quality, small portions, miserable st...."
125452,1.0,"[[], []]"
125453,1.0,"[[], []]"


En este punto, se observa que se tienen **valores vacíos** definidos como "[[], [[]". Es por ello, que resulta indispensable eliminarlos para que el modelo realice un mejor entrenamiento. Además, no se requiere el segundo parámetro de `Reviews`, recordar que se tenia tanto la reseña como la fecha de publicación. Es por ello que se procede a la eliminación de este segundo elemento.

In [ ]:
type(datos.Reviews)

pandas.core.series.Series

In [ ]:
datos.Reviews[0][3]

'J'

In [ ]:
def get_review(review):
    rev = ''
    contador_corchetes = 0

    for c in review:
        if c == '[':
            contador_corchetes += 1
        elif c == ']':
            contador_corchetes += 1
        elif contador_corchetes == 2:
            rev += c

        if contador_corchetes == 3:
            break

    return rev

In [ ]:
datos['Reviews'] = datos['Reviews'].apply(get_review)

In [ ]:
datos

,Rating,Reviews
0,5.0,"'Just like home', 'A Warm Welcome to Wintry Am..."
1,4.5,"'Great food and staff', 'just perfect'"
2,4.5,"'Satisfaction', 'Delicious old school restaurant'"
3,5.0,"'True five star dinner', 'A superb evening of ..."
4,4.5,"'Best meal.... EVER', 'super food experience'"
...,...,...
125450,1.0,
125451,1.0,"'Poor quality, small portions, miserable st...'"
125452,1.0,
125453,1.0,


In [ ]:
datos.Reviews[125452]

''

In [ ]:
datos = datos.drop(datos[datos['Reviews'] == ''].index)

In [ ]:
datos.reset_index(drop=True, inplace=True)

In [ ]:
datos

,Rating,Reviews
0,5.0,"'Just like home', 'A Warm Welcome to Wintry Am..."
1,4.5,"'Great food and staff', 'just perfect'"
2,4.5,"'Satisfaction', 'Delicious old school restaurant'"
3,5.0,"'True five star dinner', 'A superb evening of ..."
4,4.5,"'Best meal.... EVER', 'super food experience'"
...,...,...
96806,2.5,'Good Service'
96807,4.5,"'Super local eatery', 'Small and charming place'"
96808,2.0,'Cordon Bleu'
96809,2.0,"""Don't waste your time, go somewhere else!"", '..."


In [ ]:
{att : datos[datos[att].isnull()].shape[0] for att in datos.columns}

{'Rating': 0, 'Reviews': 0}

Como se ha comentado previamente, se quiere realizar analisis de sentimientos y para ello es necesario el *groud truth*. Es decir, un elemento que identifique la clase correcta de las `Reviews`, es decir, **sentimiento positivo** o **sentimieto negativo**. Al no tener el *groud truth* definido se utilizará la valoración (`Rating`).

Es por ello que la primera aproximación fué definirlo de la siguiente manera:
- Intervalo [1,3) se clasificara como sentimiento **negativo**.
- Intervalo [3] se clasificara como sentimiento **neutro**.
- Intervalo (3,5] se clasificara como sentimiento **positivo**.

Tras analizar el número de datos que se tenían de cada clasificación, se optó por omitir el "sentimiento neutro" y unificarlo con el sentimiento negativo, dado que se tenían muy pocas muestras en comparación con el positivo.

Por tanto el intervalo final queda definido como:

- Intervalo [1,3] se clasificara como sentimiento **negativo**.
- Intervalo (3,5] se clasificara como sentimiento **positivo**.

In [ ]:
datosPos = datos[datos['Rating'] > 3]
datosPos

,Rating,Reviews
0,5.0,"'Just like home', 'A Warm Welcome to Wintry Am..."
1,4.5,"'Great food and staff', 'just perfect'"
2,4.5,"'Satisfaction', 'Delicious old school restaurant'"
3,5.0,"'True five star dinner', 'A superb evening of ..."
4,4.5,"'Best meal.... EVER', 'super food experience'"
...,...,...
96782,5.0,'Friendly staff'
96791,3.5,"'These people are rude!', 'Avoid'"
96798,3.5,"'Good service', 'nice atmoshphere'"
96801,4.0,'Authentic Swiss & German Food'


In [ ]:
datosNegativo = datos[datos['Rating'] < 3]
datosNegativo

,Rating,Reviews
2820,2.5,"'This is a great place for Shoarma', 'Amsterda..."
2885,2.5,'Not the best service on a Fri. sat. or Sun......
2898,2.5,"'Boring food', 'Worst restaurant I ever had!'"
2939,2.5,"'Good local bar', 'Best Old School Hop-hip DJ'"
2977,2.5,"'Relaxed, chilled and 4 tvs!', 'Always read re..."
...,...,...
96805,2.0,"'Very amicable owner; Good pizzas', 'A very he..."
96806,2.5,'Good Service'
96808,2.0,'Cordon Bleu'
96809,2.0,"""Don't waste your time, go somewhere else!"", '..."


In [ ]:
datosNeutro = datos[datos['Rating'] == 3]
datosNeutro

,Rating,Reviews
2316,3.0,"'In general food not good in Amsterdam', 'Good..."
2568,3.0,'Good for a quick fix if you are really hun......
2577,3.0,"'Boring', 'Lovely evening sorg ny Darling.'"
2578,3.0,"'Good pizza, great deserts', 'Decent spot for ..."
2683,3.0,'A very late dinner near the Amsterdam Medi......
...,...,...
96777,3.0,'Mediocre vegetarian fried rice but rude ca......
96784,3.0,'Rude and Expensive'
96786,3.0,"'Terrible service', 'Not the wüst, not the best'"
96800,3.0,'Not just a restaurant but maybe it should...'


In [ ]:
def asignar_valor(x):
  if x >=4:
    return 1
  else:
    return 0

In [ ]:
datos['Sentimiento'] = datos['Rating'].apply(asignar_valor)

In [ ]:
datos.head(3)

,Rating,Reviews,Sentimiento
0,5.0,"'Just like home', 'A Warm Welcome to Wintry Am...",1
1,4.5,"'Great food and staff', 'just perfect'",1
2,4.5,"'Satisfaction', 'Delicious old school restaurant'",1


Ya no es necesaria la columna `Rating`, por ello se elimina. Asimismo, se añadirán dos columnas extras necesarias para el apartado 2.6 de este notebook. Esto es, separar las dos reseñas que se tienen en `Reviews` en dos columnas diferentes `Review1` y `Review2`.

In [ ]:
datos = datos.drop('Rating', axis=1)

In [ ]:
datos

,Reviews,Sentimiento
0,"'Just like home', 'A Warm Welcome to Wintry Am...",1
1,"'Great food and staff', 'just perfect'",1
2,"'Satisfaction', 'Delicious old school restaurant'",1
3,"'True five star dinner', 'A superb evening of ...",1
4,"'Best meal.... EVER', 'super food experience'",1
...,...,...
96806,'Good Service',0
96807,"'Super local eatery', 'Small and charming place'",1
96808,'Cordon Bleu',0
96809,"""Don't waste your time, go somewhere else!"", '...",0


In [ ]:
datos.Reviews[1]

"'Great food and staff', 'just perfect'"

In [ ]:
datos.Reviews[96809]

'"Don\'t waste your time, go somewhere else!", \'Horrible\''

In [ ]:
datos.Reviews[96810]

"'Poor quality, small portions, miserable st...'"

In [ ]:
def split_review(review):
    parts = review.split(", '")

    if len(parts) == 2:
        rev1 = parts[0].strip("['").strip("'")
        rev2 = parts[1].strip("']").strip("'")
    else:
        rev1 = review.strip("['").strip("'")
        rev2 = ''

    return rev1, rev2

In [ ]:
datos[['Review1', 'Review2']] = datos['Reviews'].apply(split_review).apply(pd.Series)

In [ ]:
datos

,Reviews,Sentimiento,Review1,Review2
0,"'Just like home', 'A Warm Welcome to Wintry Am...",1,Just like home,A Warm Welcome to Wintry Amsterdam
1,"'Great food and staff', 'just perfect'",1,Great food and staff,just perfect
2,"'Satisfaction', 'Delicious old school restaurant'",1,Satisfaction,Delicious old school restaurant
3,"'True five star dinner', 'A superb evening of ...",1,True five star dinner,"A superb evening of fine dining, hospitali..."
4,"'Best meal.... EVER', 'super food experience'",1,Best meal.... EVER,super food experience
...,...,...,...,...
96806,'Good Service',0,Good Service,
96807,"'Super local eatery', 'Small and charming place'",1,Super local eatery,Small and charming place
96808,'Cordon Bleu',0,Cordon Bleu,
96809,"""Don't waste your time, go somewhere else!"", '...",0,"""Don't waste your time, go somewhere else!""",Horrible


In [ ]:
datos.Review1[96809]

'"Don\'t waste your time, go somewhere else!"'

In [ ]:
datos.Review2[96809]

'Horrible'

Una vez se tienen los datos en limpio, se procede a realizar el **split** de los datos para dividir el dataset original en datos de entrenamiento, datos de validación y datos de test. Se almacenan dichos datos en `datos_entrenamiento`, `datos_validacion` y `datos_test` respectivamente. Asimismo, destacar que los datos para test no deben disponer del *ground truth*, es por ello que no se procede a concatenar dicha columna. Comentar, que se guardarán los datos limpios en memoria antes del split, en caso de que se requiera su uso.

In [ ]:
datos.to_feather("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/valoraciones_Original.feather")

In [ ]:
X = datos.drop('Sentimiento', axis=1)
y = datos['Sentimiento']

# Primero, dividimos en entrenamiento y prueba
datos_train, datos_test, groudT_train, groudT_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ahora, tienes dos conjuntos: datos_train, groudT_train para entrenamiento (posteriormente se subdividirá para obtener datos_validacion),
# y datos_test, groudT_test para prueba.

In [ ]:
datos_train.shape

(77448, 3)

In [ ]:
groudT_train.shape

(77448,)

In [ ]:
datos_test.shape

(19363, 3)

In [ ]:
groudT_test.shape

(19363,)

In [ ]:
datos_train = pd.concat([datos_train, groudT_train], axis = 1)

In [ ]:
datos_train

,Reviews,Review1,Review2,Sentimiento
58202,"'Local, Fantastic and Always Inviting', 'Reall...","Local, Fantastic and Always Inviting",Really friendly with good tapas,1
48595,"'good food,.good for lunch whilst at work'","good food,.good for lunch whilst at work",,1
66429,'Zum Dachs Restaruant Serbo-Croatian + G...',Zum Dachs Restaruant Serbo-Croatian + G...,,1
27936,'An excellent restaurant tucked away in Jon...',An excellent restaurant tucked away in Jon...,,1
89670,"'Good first Ethiopian experience', 'Ethiopian ...",Good first Ethiopian experience,Ethiopian food,1
...,...,...,...,...
6265,"'Very good', 'Real typical Tapas bar'",Very good,Real typical Tapas bar,1
54886,"'Classic', 'Dantxari - great restaurant'",Classic,Dantxari - great restaurant,1
76820,"'Cool place', 'Wonderful owner & atmosphere!'",Cool place,Wonderful owner & atmosphere!,1
860,"'Great Ribs here', 'Not perfect, but good food!'",Great Ribs here,"Not perfect, but good food!",1


###1.1.4 Comprobación de longitud mínima

En esta sección comprobamos cuál es la longitud mínima de los textos de los datos de entrenamiento a fin de conocer con mayor detalle dichos textos y hacernos una idea de qué tipo de tratamiento será necesario realizar de cara a una posible futurura tokenización y generación de *embeddings*.


In [ ]:
datos_train.Reviews.apply(lambda x: len(str(x).split())).min()

1

In [ ]:
datos_train.Reviews.apply(lambda x: len(str(x).split())).max()

20

Como se puede apreciar, parece que hay varios textos con una sola palabra. Veamos algunos de ellos:

In [ ]:
datos_train[datos_train.Reviews.apply(lambda x: len(str(x).split())) == 1]

,Reviews,Review1,Review2,Sentimiento
95036,'NICE',NICE,,1
77643,'good',good,,1
16898,'fantastic!',fantastic!,,1
62316,'NYE',NYE,,0
45418,'Wow',Wow,,1
...,...,...,...,...
45525,'scarlet',scarlet,,1
88614,'Cute',Cute,,1
25184,'yummy',yummy,,1
23483,'Gone',Gone,,0


###1.1.5 Guardado en memoria

Una vez realizada la limpieza, selección y estructuración de los datos, guardamos los mismos en ficheros `feather` para poder importarlos directamente siempre que sea necesario y no tener que ejecutar todo el proceso anterior de limpieza y estructuración siempre que queramos usar esos datos en los modelos.

Finalmente y a modo de resumen, el dataset de entrenamiento con el que tarbajarán los modelos tiene un total de 77448 registros y 4 columnas:
- `Reviews`: contenido únicamente de las dos reseñas principales del restaurante.
- `Sentimiento`: *groud truth*, elemento que identifica el sentimiento correcto (1 positivo, 0 negativo).
- `Review1`: contenido únicamente de la primera reseña realizada al restaurante dado.
- `Review2`: contenido únicamente de la segunda reseña realizada al restaurante dado.


Por otro lado, el dataset de validación que se genera, tiene la misma estructura pero cuenta solo con 19363 registros.

Antes de guardar los datos de entrenamiento en el fichero correspondiente, se ha procedido a resetear los índices de las muestras para evitar que estos den problemas en caso de que, al haber eliminado, reestructurado o limpiado muestras, se haya generado alguna discrepancia o hueco en los mismos y estos hayan dejado de ser consecutivos.

In [ ]:
datos_train.reset_index(drop=True, inplace=True)

In [ ]:
datos_train.head(3)

,Reviews,Review1,Review2,Sentimiento
0,"'Local, Fantastic and Always Inviting', 'Reall...","Local, Fantastic and Always Inviting",Really friendly with good tapas,1
1,"'good food,.good for lunch whilst at work'","good food,.good for lunch whilst at work",,1
2,'Zum Dachs Restaruant Serbo-Croatian + G...',Zum Dachs Restaruant Serbo-Croatian + G...,,1


In [ ]:
datos_train.shape

(77448, 4)

In [ ]:
datos_train.to_feather("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/valoraciones_Original_train.feather")

In [ ]:
datos_test.reset_index(drop=True, inplace=True)

In [ ]:
datos_test.head(3)

,Reviews,Review1,Review2
0,"'Unfriendly staff, no pets allowed', 'Average ...","Unfriendly staff, no pets allowed",Average Slovak food
1,"'Christmas Break', 'Cocktails and music'",Christmas Break,Cocktails and music
2,"'Interesting type of pizza', 'Best trancio ever.'",Interesting type of pizza,Best trancio ever.


In [ ]:
datos_test.shape

(19363, 3)

In [ ]:
datos_test.to_feather("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/valoraciones_Original_test.feather")

##1.2 Datos divididos

Con este esquema de datos, se pretende separar las dos reseñas que se proporcionan sobre cada restaurante. Con esto, se obtiene un conjunto de datos mayor para entrenamiento lo que podría mejorar el resultado del modelo. Este conjunto de datos, se ulitilzará posteriormente en el apartado 2.5. Asimismo, en este apartado se partirá del conjunto de datos ya limpio obtenido en la sección 1.1.

###1.2.1 Imports

In [ ]:
import numpy as np
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split

In [ ]:
warnings.filterwarnings("ignore")

###1.2.2 Carga de datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
datos_paraDividir = pd.read_feather("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/valoraciones_Original.feather")
datos_paraDividir

,Reviews,Sentimiento,Review1,Review2
0,"'Just like home', 'A Warm Welcome to Wintry Am...",1,Just like home,A Warm Welcome to Wintry Amsterdam
1,"'Great food and staff', 'just perfect'",1,Great food and staff,just perfect
2,"'Satisfaction', 'Delicious old school restaurant'",1,Satisfaction,Delicious old school restaurant
3,"'True five star dinner', 'A superb evening of ...",1,True five star dinner,"A superb evening of fine dining, hospitali..."
4,"'Best meal.... EVER', 'super food experience'",1,Best meal.... EVER,super food experience
...,...,...,...,...
96806,'Good Service',0,Good Service,
96807,"'Super local eatery', 'Small and charming place'",1,Super local eatery,Small and charming place
96808,'Cordon Bleu',0,Cordon Bleu,
96809,"""Don't waste your time, go somewhere else!"", '...",0,"""Don't waste your time, go somewhere else!""",Horrible


In [ ]:
datos_paraDividir.Review2[96808]

''

###1.2.3 División

In [ ]:
datos_divididos = datos_paraDividir.copy()

In [ ]:
datos_divididos = datos_divididos.melt(id_vars=['Sentimiento'], value_vars=['Review1', 'Review2'], value_name='Reviews').drop(columns=['variable'])

In [ ]:
datos_divididos

,Sentimiento,Reviews
0,1,Just like home
1,1,Great food and staff
2,1,Satisfaction
3,1,True five star dinner
4,1,Best meal.... EVER
...,...,...
193617,0,
193618,1,Small and charming place
193619,0,
193620,0,Horrible


Anteriormente, se vió que se tenían datos vacíos en `Review2`, esto era proque se tenían reseñas unicamente de una persona y no de dos como se comenta en Kaggel. Por ello, pasar de 96811 filas a 193622 (el doble), se tienen que haber añadido filas vacías, observar *id 193617*. Se procederá a su eliminación dado que no aportan ningún tipo de información.

In [ ]:
def revisar_espacios_en_blanco(review):
   if review.strip() == '':
        return np.nan
   else:
        return review.strip()

datos_divididos['Reviews'] = datos_divididos['Reviews'].apply(revisar_espacios_en_blanco)

valores_espacios_en_blanco = (datos_divididos['Reviews'] == '').sum()

valores_nulos = datos_divididos.isnull().sum()

print(f"Espacios en blanco en Reviews: {valores_espacios_en_blanco}")
print(valores_nulos)


Espacios en blanco en Reviews: 0
Sentimiento        0
Reviews        15162
dtype: int64


In [ ]:
datos_divididos = datos_divididos.dropna(subset=['Reviews'])

valores_espacios_en_blanco = (datos_divididos['Reviews'] == '').sum()
valores_nulos = datos_divididos.isnull().sum()

print(f"Espacios en blanco en Reviews después de eliminar: {valores_espacios_en_blanco}")
print(valores_nulos)


Espacios en blanco en Reviews después de eliminar: 0
Sentimiento    0
Reviews        0
dtype: int64


In [ ]:
datos_divididos

,Sentimiento,Reviews
0,1,Just like home
1,1,Great food and staff
2,1,Satisfaction
3,1,True five star dinner
4,1,Best meal.... EVER
...,...,...
193613,0,"It was really horrible, I would never go a..."
193614,0,38 USD for a pizza!! HORRIBLE PRICING!
193616,0,A very helpful and good Sri Lankan attenda...
193618,1,Small and charming place


Al igual que en la sección 1.1, se procederá a dividir (**split**) tanto en conjunto de train como conjunto de test.

In [ ]:
X = datos_divididos.drop('Sentimiento', axis=1)
y = datos_divididos['Sentimiento']

# Primero, dividimos en entrenamiento y prueba
datos_divididos_train, datos_divididos_test, groudT_divididos_train, groudT_divididos_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ahora, tienes dos conjuntos: datos_train, groudT_train para entrenamiento (posteriormente se subdividirá para obtener datos_validacion),
# y datos_test, groudT_test para prueba.

In [ ]:
datos_divididos_train.shape

(142768, 1)

In [ ]:
groudT_divididos_train.shape

(142768,)

In [ ]:
datos_divididos_test.shape

(35692, 1)

In [ ]:
groudT_divididos_test.shape

(35692,)

In [ ]:
datos_divididos_train = pd.concat([datos_divididos_train, groudT_divididos_train], axis = 1)

In [ ]:
datos_divididos_train

,Reviews,Sentimiento
26278,A most friendly welcome,1
59100,Nice treat,0
56853,"Lovely interior design', ""Don't let the lack o...",1
117791,Nice Food and Good Service,1
22714,Just ok,0
...,...,...
119879,Great Chinese food,0
103694,Take a detour off the main drag,1
131932,Good Burrito,1
146867,Super Rude,0


###1.2.4 Comprobación de longitud

Del mismo modo que se realizó para el apartado 1.1.4, se comprobará cuál es la longitud mínima de los textos de los datos de entrenamiento. Esto se realiza para tener un mayor conocimiento sobre los datos.

In [ ]:
datos_divididos_train.Reviews.apply(lambda x: len(str(x).split())).min()

1

In [ ]:
datos_divididos_train.Reviews.apply(lambda x: len(str(x).split())).max()

19

In [ ]:
datos_divididos_train[datos_divididos.Reviews.apply(lambda x: len(str(x).split())) == 1]

,Reviews,Sentimiento
173745,Brunch,1
42672,Lunch,0
131903,Average,0
98031,Delicious,1
129451,Good,1
...,...,...
69092,Superburgers!,1
66803,Friendly,1
123855,Interesting.,1
54886,Classic,1


###1.2.5 Guardado en memoria

Finalmente, una vez realizada la limpieza, y estructuración de los datos, guardamos los mismos en ficheros `feather` para poder importarlos directamente siempre que sea necesario y no tener que ejecutar todo el proceso anterior de nuevo.

A modo de resumen, el dataset de entrenamiento con el que tarbajarán los modelos en el apartado 2.5, tiene un total de 142768 registros (aproximadamente el doble que antes) y 2 columnas:
- `Reviews`: contenido únicamente de las dos reseñas principales del restaurante.
- `Sentimiento`: *groud truth*, elemento que identifica el sentimiento correcto (1 positivo, 0 negativo).

Por otro lado, el dataset de validación que se genera, tiene la misma estructura pero cuenta solo con 35692 registros (recordar que este no contiene los valores de la columna `Sentimiento`).

In [ ]:
datos_divididos_train.reset_index(drop=True, inplace=True)

In [ ]:
datos_divididos_train.head(3)

,Reviews,Sentimiento
0,A most friendly welcome,1
1,Nice treat,0
2,"Lovely interior design', ""Don't let the lack o...",1


In [ ]:
datos_divididos_train.shape

(142768, 2)

In [ ]:
datos_divididos_train.to_feather("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/valoraciones_Separados_train.feather")

In [ ]:
datos_divididos_test.reset_index(drop=True, inplace=True)

In [ ]:
datos_divididos_test.head(3)

,Reviews
0,"perfecto, italiano restaurant"
1,Great quality for a very honest price.
2,"Friendly & Yummy', ""The best falafel I've ever..."


In [ ]:
datos_divididos_test.shape

(35692, 1)

In [ ]:
datos_divididos_test.to_feather("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/valoraciones_Separados_test.feather")

##1.3 Datos Siamese

###1.3.1 Imports

In [ ]:
import numpy as np
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split

In [ ]:
warnings.filterwarnings("ignore")

###1.3.2 Carga de datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
datos_siamese = pd.read_feather("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/valoraciones_Original.feather")
datos_siamese

,Reviews,Sentimiento,Review1,Review2
0,"'Just like home', 'A Warm Welcome to Wintry Am...",1,Just like home,A Warm Welcome to Wintry Amsterdam
1,"'Great food and staff', 'just perfect'",1,Great food and staff,just perfect
2,"'Satisfaction', 'Delicious old school restaurant'",1,Satisfaction,Delicious old school restaurant
3,"'True five star dinner', 'A superb evening of ...",1,True five star dinner,"A superb evening of fine dining, hospitali..."
4,"'Best meal.... EVER', 'super food experience'",1,Best meal.... EVER,super food experience
...,...,...,...,...
96806,'Good Service',0,Good Service,
96807,"'Super local eatery', 'Small and charming place'",1,Super local eatery,Small and charming place
96808,'Cordon Bleu',0,Cordon Bleu,
96809,"""Don't waste your time, go somewhere else!"", '...",0,"""Don't waste your time, go somewhere else!""",Horrible


In [ ]:
datos_siamese.Review2[96808]

''

###1.3.3 Siamese

En este caso, se requieren dos reseñas para comparar sus embeddings, es decir, comparar la similitud semántica entre los textos. Es por ello, que ya no se requiere la columna `Reviews` y se procede a eliminar.

In [ ]:
def revisar_espacios_en_blanco(review):
   if review.strip() == '':
        return np.nan
   else:
        return review.strip()

datos_siamese['Review2'] = datos_siamese['Review2'].apply(revisar_espacios_en_blanco)

In [ ]:
datos_siamese = datos_siamese.dropna(subset=['Review2'])

In [ ]:
datos_siamese = datos_siamese.drop('Reviews', axis=1)

In [ ]:
datos_siamese

,Sentimiento,Review1,Review2
0,1,Just like home,A Warm Welcome to Wintry Amsterdam
1,1,Great food and staff,just perfect
2,1,Satisfaction,Delicious old school restaurant
3,1,True five star dinner,"A superb evening of fine dining, hospitali..."
4,1,Best meal.... EVER,super food experience
...,...,...,...
96802,0,Horrible!,"It was really horrible, I would never go a..."
96803,0,Good service,38 USD for a pizza!! HORRIBLE PRICING!
96805,0,Very amicable owner; Good pizzas,A very helpful and good Sri Lankan attenda...
96807,1,Super local eatery,Small and charming place


Al igual que en la sección 1.1, se procederá a dividir (**split**) tanto en conjunto de train como conjunto de test.

In [ ]:
X = datos_siamese.drop('Sentimiento', axis=1)
y = datos_siamese['Sentimiento']

# Primero, dividimos en entrenamiento y prueba
datos_siamese_train, datos_siamese_test, groudT_siamese_train, groudT_siamese_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ahora, tienes dos conjuntos: datos_train, groudT_train para entrenamiento (posteriormente se subdividirá para obtener datos_validacion),
# y datos_test, groudT_test para prueba.

In [ ]:
datos_siamese_train.shape

(65319, 2)

In [ ]:
groudT_siamese_train.shape

(65319,)

In [ ]:
datos_siamese_test.shape

(16330, 2)

In [ ]:
groudT_siamese_test.shape

(16330,)

In [ ]:
datos_siamese_train = pd.concat([datos_siamese_train, groudT_siamese_train], axis = 1)

In [ ]:
datos_siamese_train

,Review1,Review2,Sentimiento
60784,Nice brunch,Good place,1
531,Worlds best hot cocolate!,Great place!,1
62256,Great food and service,"Rated on 7,0!",1
80106,Delicious and friendly,Great little French Bistro!,0
77020,Late dinner,Very nice dinner,0
...,...,...,...
6861,"Very nice food, great location",Great calamari,1
64642,Great Bavarian food in a cozy place,Traditional restaurant,1
90931,Italy traditions in Vienna,Very nice restaurant,1
873,3 visits over 4 days,Great food & atmosphere,1


###1.3.4 Comprobación de longitud

A fin de tener mejor entendimiento de los datos para su posterior uso, se comprueba la longitud que te tiene de cada par de reseñas.

In [ ]:
datos_siamese_train.Review1.apply(lambda x: len(str(x).split())).min()

1

In [ ]:
datos_siamese_train.Review1.apply(lambda x: len(str(x).split())).max()

12

In [ ]:
datos_siamese_train.Review2.apply(lambda x: len(str(x).split())).min()

1

In [ ]:
datos_siamese_train.Review2.apply(lambda x: len(str(x).split())).max()

12

In [ ]:
datos_siamese_train[datos_siamese_train.Review1.apply(lambda x: len(str(x).split())) == 1]

,Review1,Review2,Sentimiento
56780,Excellent,Try the Special Fixed Price Dinner,1
8533,Fantastic,Delicious quick serve food,1
36647,Sublime,Outstanding,1
95293,No,Great place for family !,1
13196,Great,Nothing special,1
...,...,...,...
2964,McDonalds,Deal with it.,0
79734,Lunch,Pleasantly surprised !,0
56064,Satisfied,Lots of meat,1
47725,Empty,Good burgers but overpriced!,1


In [ ]:
datos_siamese_train[datos_siamese_train.Review2.apply(lambda x: len(str(x).split())) == 1]

,Review1,Review2,Sentimiento
33846,"very simple , but good quality sushi",Sushi?,1
43886,Wonderful Wedding venue,Date,1
20961,A bar with a history !,Crazy....,1
84469,The best breakfast/lunch,Super,1
36647,Sublime,Outstanding,1
...,...,...,...
61942,A Good Japanese Restaurant,Testwr,1
41447,Bacon Jam is magic,BOOM!!,1
80526,Great burguer,Finest,1
90575,"""Wonderful Kaffe""",Recomends,0


###1.3.5 Guardado en memoria

Una vez realizada la limpieza, y estructuración de los datos, se guardan en ficheros `feather`dichos datos para poder importarlos directamente siempre que sea necesario y no tener que ejecutar todo el proceso anterior de nuevo.

A modo de resumen, el dataset de entrenamiento con el que tarbajarán los modelos en el apartado 2.6, tiene un total de 65319 registros (12000 datos menos que la implementación con el conjunto de datos originales) y 3 columnas:
- `Sentimiento`: *groud truth*, elemento que identifica el sentimiento correcto (1 positivo, 0 negativo).
- `Review1`: contenido únicamente de la primera reseña realizada al restaurante dado.
- `Review2`: contenido únicamente de la segunda reseña realizada al restaurante dado.

Por otro lado, el dataset de validación que se genera, tiene la misma estructura pero cuenta solo con 16330 registros.

In [ ]:
datos_siamese_train.reset_index(drop=True, inplace=True)

In [ ]:
datos_siamese_train.head(3)

,Review1,Review2,Sentimiento
0,Nice brunch,Good place,1
1,Worlds best hot cocolate!,Great place!,1
2,Great food and service,"Rated on 7,0!",1


In [ ]:
datos_siamese_train.shape

(65319, 3)

In [ ]:
datos_siamese_train.to_feather("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/valoraciones_Siamese_train.feather")

In [ ]:
datos_siamese_test.reset_index(drop=True, inplace=True)

In [ ]:
datos_siamese_test.head(3)

,Review1,Review2
0,Lovely food!,Nightmare
1,"Fish is excellent, not enough veggies",Avarege
2,Very good lunch place,Terrible!!


In [ ]:
datos_siamese_test.shape

(16330, 2)

In [ ]:
datos_siamese_test.to_feather("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/valoraciones_Siamese_test.feather")

#2. Modelos

En esta sección se presentan las distintas arquitecturas y aproximaciones definidas para intentar dar solución al problema que nos atañe: ***sentiment analysis***.




###2.1 Imports Generales

In [ ]:
import numpy as np
import pandas as pd
import random
import os
import warnings

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn import metrics
import matplotlib.pyplot as plt

###2.2 Imports modelos y demás

In [ ]:
!pip install transformers

In [ ]:
!pip install torch

In [ ]:
!pip install sentencepiece

In [ ]:
# Imports transformers
import torch
import transformers
from transformers import BertTokenizer, BertModel
from transformers import RobertaTokenizer, RobertaModel
from transformers import AutoConfig
from transformers import AutoModel
from transformers import AutoTokenizer
import torch.nn.functional as F

# Imports utilidades
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [ ]:
# Para que los modelos no inunden de información irreleavante las salidas.
# Si no se ejecuta esta celda cada vez que se trunque un texto saldrá una notificiación.
transformers.logging.set_verbosity_error()

##2.3 Definición de variables generales

Variables definidas y su uso:
* `MAX_LEN`: Tamaño máximo de los subconjuntos de textos que se pasan a los modelos.
* `TRAIN_BATCH_SIZE`: Cantidad de muestras con las que se entrena el modelo cada vez. Los valores máximos que soportan nuestras capacidades de computación son entre 8 y 16 según el modelo. Con valores mayores se corre el riesgo de que la sesión de entrenamiento desborde la memoria y colapse.
* `VALID_BATCH_SIZE`: Idéntico al `TRAIN_BATCH_SIZE` pero para la etapa de testeo y validación. En este caso, al no haber tantos cálculos durante la validación como durante el entrenamiento, soporta valores mayores.
* `EPOCHS`: Número de veces que se quieren entrenar los modelos con el conjunto de datos. Ej: Con 2 epochs el modelo entrenará 2 veces con cada muestra del dataset.
* `LEARNING_RATE`: Modificación de la velocidad con la que aprende el modelo.
* `TRAIN_SIZE`: Porcentaje de los datos que se usa para el entrenamiento de los modelos. Los datos sobrantes se reservan para el testeo de los modelos y para la obtención de métricas de la calidad de los mismos.
* `PASOS_POR_INTERVALO`: Número que indica cada cuantos batches mostrar información relativa a las métricas del modelo en entrenamiento.

In [ ]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 64
EPOCHS = 1
LEARNING_RATE = 1e-7
#TRAIN_SIZE = 0.3
TRAIN_SIZE = 0.5
#TRAIN_SIZE = 0.8
PASOS_POR_INTERVALO = 10

Dadas las limitaciones en la capacidad de cómputo, se han empleado 3 valores distintos para el parámetro `TRAIN_SIZE`:
1. `TRAIN_SIZE = 0.3`: Inicialmente y dado que los entrenamientos con más datos llevaban bastante tiempo, se entrenaron algunos modelos con solo el 30% de las muestras para agilizar dicho entrenamiento y tener unas primeras señales de cómo se desenvolvían.
1. `TRAIN_SIZE = 0.5`: Se trata de un punto de equilibrio entre usar pocos datos y ser rápidos o usar más con (posiblemente) mejores resultados pero a costa de velocidad. La mayoría de modelos han sido entrenados con este porcentaje de datos por esta misma razón.
1. `TRAIN_SIZE = 0.8`: Esta proporción de datos se probó a fin de ver si, con una mayor cantidad de datos, se conseguían mejores resultados. Como los resultados obtenidos fueron similares a los conseguidos con el 50% y como el tiempo de entrenamiento era mayor, se decidió no proseguir con esta aproximación.


##2.4 Aproximación 1: Con los datos originales

El valor por defecto de los hiperparámetros que se han utilizado:
- MAX_LEN = 512
- TRAIN_BATCH_SIZE = 8
- VALID_BATCH_SIZE = 64
- EPOCHS = 1
- LEARNING_RATE = 1e-5
- TRAIN_SIZE = 0.5
- PASOS_POR_INTERVALO = 10

###2.4.1 Carga de datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
datos = pd.read_feather("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/valoraciones_Original_train.feather")
datos

,Reviews,Review1,Review2,Sentimiento
0,"'Local, Fantastic and Always Inviting', 'Reall...","Local, Fantastic and Always Inviting",Really friendly with good tapas,1
1,"'good food,.good for lunch whilst at work'","good food,.good for lunch whilst at work",,1
2,'Zum Dachs Restaruant Serbo-Croatian + G...',Zum Dachs Restaruant Serbo-Croatian + G...,,1
3,'An excellent restaurant tucked away in Jon...',An excellent restaurant tucked away in Jon...,,1
4,"'Good first Ethiopian experience', 'Ethiopian ...",Good first Ethiopian experience,Ethiopian food,1
...,...,...,...,...
77443,"'Very good', 'Real typical Tapas bar'",Very good,Real typical Tapas bar,1
77444,"'Classic', 'Dantxari - great restaurant'",Classic,Dantxari - great restaurant,1
77445,"'Cool place', 'Wonderful owner & atmosphere!'",Cool place,Wonderful owner & atmosphere!,1
77446,"'Great Ribs here', 'Not perfect, but good food!'",Great Ribs here,"Not perfect, but good food!",1


###2.4.2 Definición tokenizadores y modelos

Para la tarea de analisis de sentimientos que se prentende llevar a cabo, se ha decido utilizar *transformers* preentrenados para la generación de los *embeddings* de los textos y acoplarles nosotros una cabeza a entrenar específicamente diseñada para la tarea que nos atañe, es decir, clasificación de una reseña en función de si ha sido positiva o negativa.

Para el diseño de las cabezas de clasificación se ha seguido la siguiente aproximación:
- Utilización de redes neuronales densas.

Los distintos *transformers* que se han empleado para la generación de los *embeddings* han sido:
- BERT
- RoBERTa



In [ ]:
tokenizerB = BertTokenizer.from_pretrained("bert-base-uncased")
modelB = BertModel.from_pretrained("bert-base-uncased")
configB = AutoConfig.from_pretrained("bert-base-uncased")

In [ ]:
tokenizerRB = RobertaTokenizer.from_pretrained("roberta-base")
modelRB = RobertaModel.from_pretrained("roberta-base")
configRB = AutoConfig.from_pretrained("roberta-base")

###2.4.3 Función de generación de conjuntos de train y test: DataLoaders

Esta función se encarga de generar los *DataLoaders* de entrenamiento y test. Un DataLoader es una herramienta que facilita la carga y gestión de datos durante el entrenamiento de modelos de aprendizaje automático. Es decir, cargar datos de manera eficiente, particionarlos en lotes (batches) y proporcionarlos continuamente al modelo durante el entrenamiento/test.

Sus parámetros son:
- `dataframe`: El *DataFrame* que contiene la información con la que se quiere hacer los *DataLoaders*.
- `tokenizer`: El tokenizador que se utilizará para tokenizar los textos.
- `dataset`: La clase *CustomDataset* con la que se definirán los conjuntos de datos.
- `collator`: La clase *DataCollator* que se encargará de gestionar los *batches* en cada uno de los *DataLoader*.

In [ ]:
def generate_loaders(dataframe, tokenizer, dataset, collator):
  train_dataset = dataframe.sample(frac=TRAIN_SIZE, random_state=0)
  test_dataset = dataframe.drop(train_dataset.index).reset_index(drop=True)
  train_dataset = train_dataset.reset_index(drop=True)

  print(f"FULL Dataset:{dataframe.shape}")
  print(f"TRAIN Dataset: {train_dataset.shape}")
  print(f"TEST Dataset: {test_dataset.shape}")

  training_set = dataset(train_dataset, MAX_LEN)
  testing_set = dataset(test_dataset, MAX_LEN)
  dc = collator(tokenizer, MAX_LEN)


  train_params = {'batch_size': TRAIN_BATCH_SIZE,
                  'shuffle': True,
                  'num_workers': 0,
                  'collate_fn': dc
                  }

  test_params = { 'batch_size': VALID_BATCH_SIZE,
                  'shuffle': True,
                  'num_workers': 0,
                  'collate_fn': dc
                }

  training_loader = DataLoader(training_set, **train_params)
  testing_loader = DataLoader(testing_set, **test_params)
  return training_loader, testing_loader

###2.4.4 Single transform approach

En esta aproximación se plantea una arquitectura para la obtención de *embeddings* con los que entrenará la cabeza de clasificación.

####2.4.4.1 Definición restaurantsDataset y DataCollatorRestaurant

Esta clase es la encargada de elaborar las muestras con las que van a entrenar los modelos.

Cada muestra consiste en:
- La etiqueta *sentimiento*, que representa el valor de la columna `Sentimineto` de los datos, es decir, si el sentimiento es positivo o negativo.
- `Reviews`, comentarios que se utilizan para entrenar el modelo.

En esta aproximación se busca generar el *embedding* de las reseñas.

In [ ]:
type(datos)

pandas.core.frame.DataFrame

In [ ]:
datos

,Reviews,Review1,Review2,Sentimiento
0,"'Local, Fantastic and Always Inviting', 'Reall...","Local, Fantastic and Always Inviting",Really friendly with good tapas,1
1,"'good food,.good for lunch whilst at work'","good food,.good for lunch whilst at work",,1
2,'Zum Dachs Restaruant Serbo-Croatian + G...',Zum Dachs Restaruant Serbo-Croatian + G...,,1
3,'An excellent restaurant tucked away in Jon...',An excellent restaurant tucked away in Jon...,,1
4,"'Good first Ethiopian experience', 'Ethiopian ...",Good first Ethiopian experience,Ethiopian food,1
...,...,...,...,...
77443,"'Very good', 'Real typical Tapas bar'",Very good,Real typical Tapas bar,1
77444,"'Classic', 'Dantxari - great restaurant'",Classic,Dantxari - great restaurant,1
77445,"'Cool place', 'Wonderful owner & atmosphere!'",Cool place,Wonderful owner & atmosphere!,1
77446,"'Great Ribs here', 'Not perfect, but good food!'",Great Ribs here,"Not perfect, but good food!",1


In [ ]:
class restaurantsDataset(Dataset):
  def __init__(self, dataframe, max_len):
    self.data = dataframe
    self.comentario = dataframe.Reviews
    self.targets = self.data.Sentimiento
    self.max_len = max_len

  def __len__(self):
    return len(self.comentario)

  def __getitem__(self, index):
    comment = self.comentario[index]
    target = self.targets[index]

    return {
        "comentario": comment,
        "target": target
    }

In [ ]:
datasetRestaurants = restaurantsDataset(datos, MAX_LEN)
salida = datasetRestaurants.__getitem__(0)
salida

{'comentario': "'Local, Fantastic and Always Inviting', 'Really friendly with good tapas'",
 'target': 1}

In [ ]:
aux = encoder_input = tokenizerB(
          salida["comentario"],
          None,
          add_special_tokens=True,
          truncation=True,
          max_length=MAX_LEN,
          pad_to_max_length=True,
          return_tensors = "pt",
          return_token_type_ids=True
          )
aux["input_ids"]

tensor([[  101,  1005,  2334,  1010, 10392,  1998,  2467, 15085,  1005,  1010,
          1005,  2428,  5379,  2007,  2204, 11112,  3022,  1005,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [ ]:
class DataCollatorRestaurant:
    def __init__(self, tokenizer, max_len=512):
      self.tokenizer = tokenizer
      self.max_len = max_len

    def __call__(self, input_batch):
      data_frame = pd.DataFrame(input_batch)
      batch_dict = {column: data_frame[column].tolist() for column in data_frame}

      encoder_input = self.tokenizer(
          batch_dict["comentario"],
          None,
          add_special_tokens=True,
          truncation=True,
          max_length=self.max_len,
          pad_to_max_length=True,
          return_tensors = "pt",
          return_token_type_ids=True
          )

      return {
      'ids': encoder_input["input_ids"],
      'mask': encoder_input["attention_mask"],
      "target": torch.Tensor(batch_dict["target"]),
      "token_type_ids": encoder_input["token_type_ids"]
    }

####2.4.4.2 Modelo 1: BERT

##### Definición Dataset y Dataloader

Empleando la función anteriormente desarrollada, podemos generar los *loaders* sin tener que especificar nada más que el *tokenizer* concreto y el *Dataset* y el *DataCollator*.

In [ ]:
training_loader, testing_loader = generate_loaders(datos,
                                                   tokenizerB,
                                                   restaurantsDataset,
                                                   DataCollatorRestaurant
                                                   )

FULL Dataset:(77448, 4)
TRAIN Dataset: (23234, 4)
TEST Dataset: (54214, 4)


##### Definición del modelo

In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.dropout = 0.2
        self.hidden_embd = 768
        self.output_layer = 1

        # Layers
        self.l1 = BertModel.from_pretrained('bert-base-uncased')
        #self.l2 = torch.nn.Linear(self.hidden_embd, 256)
        #self.l3 = torch.nn.Linear(256, 64)
        self.l4 = torch.nn.Dropout(self.dropout)
        # self.l5 = torch.nn.Linear(64, self.output_layer)
        self.l5 = torch.nn.Linear(self.hidden_embd, self.output_layer)
        self.l6 = torch.nn.Sigmoid()

    def forward(self, ids, mask, token_type_ids):
        _, output_1 = self.l1(ids, attention_mask=mask, token_type_ids=token_type_ids, return_dict=False)
        #output_2 = self.l2(output_1)
        #output_3 = self.l3(output_2)
        output_4 = self.l4(output_1)
        output_5 = self.l5(output_4)
        output = self.l6(output_5)
        return output

model = BERTClass()
model.to(device)

BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE,
                             weight_decay=0.01)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0.01
)

##### Función de entrenamiento


In [ ]:
def trainBert(epoch):
  model.train()
  loss_fn = torch.nn.BCEWithLogitsLoss()
  num_iteraciones = len(training_loader)
  sum_loss = 0

  for iteracion, data in enumerate(training_loader, 0):
    ids = data['ids'].to(device)
    mask = data['mask'].to(device)
    targets = data['target'].to(device)
    token_type_ids = data["token_type_ids"].to(device)

    output = model(ids, mask, token_type_ids)
    optimizer.zero_grad()

    perdida = loss_fn(output.squeeze(), targets)
    with torch.no_grad():
      sum_loss+=perdida
      if iteracion % PASOS_POR_INTERVALO ==0:
        print(f'Epoch: {epoch}, iteración; {iteracion} de {num_iteraciones}, Loss: {sum_loss.cpu().numpy()/PASOS_POR_INTERVALO}')
        sum_loss = 0

    optimizer.zero_grad()
    perdida.backward()
    optimizer.step()

##### Función de entrenamiento v2


In [ ]:
train_loss_history = []
train_accuracy_history = []
val_loss_history = []
val_accuracy_history = []

In [ ]:
def trainBert(epoch, model, training_loader, optimizer, device):
    model.train()
    loss_fn = torch.nn.BCEWithLogitsLoss()
    num_iteraciones = len(training_loader)
    sum_loss = 0
    correct_predictions = 0
    total_predictions = 0

    for iteracion, data in enumerate(training_loader, 0):
        ids = data['ids'].to(device)
        mask = data['mask'].to(device)
        targets = data['target'].to(device)
        token_type_ids = data["token_type_ids"].to(device)

        output = model(ids, mask, token_type_ids)
        optimizer.zero_grad()

        # Calculate loss and update sum_loss
        perdida = loss_fn(output.squeeze(), targets)
        sum_loss += perdida.item()

        # Calculate accuracy
        predictions = torch.sigmoid(output).cpu().numpy() >= 0.5
        correct_predictions += (predictions == targets.cpu().numpy()).sum()
        total_predictions += targets.size(0)

        # Your existing code for backward pass and optimizer step
        if iteracion % PASOS_POR_INTERVALO == 0:
                print(f'Epoch: {epoch}, iteración: {iteracion} de {num_iteraciones}, Loss: {sum_loss / PASOS_POR_INTERVALO}')
                accuracy = metrics.accuracy_score(targets.cpu().numpy(), (torch.sigmoid(output).cpu().numpy() >= 0.5))

        optimizer.zero_grad()
        perdida.backward()
        optimizer.step()

    # Calculate average loss and accuracy for the epoch
    epoch_loss = sum_loss / num_iteraciones
    epoch_accuracy = correct_predictions / total_predictions

    # Append to history
    train_loss_history.append(epoch_loss)
    train_accuracy_history.append(epoch_accuracy)

    return epoch_loss, epoch_accuracy

##### Entrenamiento del modelo (30% de datos)

Utiliza:
- LR 1e-5
- weight_decay=0.01
- Dropout + Linear + Sigmoid

In [ ]:
for epoch in range(EPOCHS):
  trainBert(epoch)

Epoch: 0, iteración; 0 de 1453, Loss: 0.057828837633132936
Epoch: 0, iteración; 10 de 1453, Loss: 0.6029022216796875
Epoch: 0, iteración; 20 de 1453, Loss: 0.6289915561676025
Epoch: 0, iteración; 30 de 1453, Loss: 0.6424582481384278
Epoch: 0, iteración; 40 de 1453, Loss: 0.6494301319122314
Epoch: 0, iteración; 50 de 1453, Loss: 0.612917137145996
Epoch: 0, iteración; 60 de 1453, Loss: 0.5878406524658203
Epoch: 0, iteración; 70 de 1453, Loss: 0.6408910274505615
Epoch: 0, iteración; 80 de 1453, Loss: 0.6027796745300293
Epoch: 0, iteración; 90 de 1453, Loss: 0.5816263198852539
Epoch: 0, iteración; 100 de 1453, Loss: 0.6241870880126953
Epoch: 0, iteración; 110 de 1453, Loss: 0.6019627094268799
Epoch: 0, iteración; 120 de 1453, Loss: 0.6104374408721924
Epoch: 0, iteración; 130 de 1453, Loss: 0.5722097396850586
Epoch: 0, iteración; 140 de 1453, Loss: 0.5774763107299805
Epoch: 0, iteración; 150 de 1453, Loss: 0.6381230354309082
Epoch: 0, iteración; 160 de 1453, Loss: 0.5934973239898682
Epoch: 

In [ ]:
try:
    torch.save(model.state_dict(), "./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_30.pth")
    print("Modelo guardado correctamente")
except Exception as e:
    print(f"Error al guardar el modelo: {e}")

Modelo guardado correctamente


##### Evaluación del modelo

In [ ]:
def validationBert(epoch):
  model.eval()
  fin_targets=[]
  fin_outputs=[]
  num_iteraciones = len(testing_loader)
  with torch.no_grad():
    for i, data in enumerate(testing_loader, 0):
      ids = data['ids'].to(device)
      mask = data['mask'].to(device)
      token_type_ids = data['token_type_ids'].to(device)
      targets = data['target'].to(device)

      print(f"Batch {i} de {num_iteraciones}")

      outputs = model(ids, mask, token_type_ids)
      fin_targets.extend(targets.cpu().detach().numpy().tolist())
      fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
  return fin_outputs, fin_targets

**Nota:** Esta celda no se encuentra ejecutada porque solo se usa cuando la evaluación de un modelo se ha hecho en una sesión distinta a su entrenamiento. Si se hace en la misma sesión, no tenemos necesidad de cargar el modelo (que acabamos de entrenar). Por el contrario, si lo entrenamos "hoy" y lo evaluamos "mañana", en vez de reentrenarlo para poder evaluarlo, solo es necesario cargarlo.

In [ ]:
if os.path.exists("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_30.pth"):
  model = BERTClass()
  if device == "cuda":
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_30.pth"))
    print("Modelo cargado correctamente")
  else:
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_30.pth", map_location="cpu"))
    print("Modelo cargado correctamente con CPU")

  model.eval()
  model.to(device)

Modelo cargado correctamente


In [ ]:
for epoch in range(EPOCHS):
  outputs, targets = validationBert(epoch)
  outputs = np.array(outputs) >= 0.5

  targets = np.array(targets).flatten().astype(int)

  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
  print(f"Accuracy Score = {accuracy}")
  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

Batch 0 de 848
Batch 1 de 848
Batch 2 de 848
Batch 3 de 848
Batch 4 de 848
Batch 5 de 848
Batch 6 de 848
Batch 7 de 848
Batch 8 de 848
Batch 9 de 848
Batch 10 de 848
Batch 11 de 848
Batch 12 de 848
Batch 13 de 848
Batch 14 de 848
Batch 15 de 848
Batch 16 de 848
Batch 17 de 848
Batch 18 de 848
Batch 19 de 848
Batch 20 de 848
Batch 21 de 848
Batch 22 de 848
Batch 23 de 848
Batch 24 de 848
Batch 25 de 848
Batch 26 de 848
Batch 27 de 848
Batch 28 de 848
Batch 29 de 848
Batch 30 de 848
Batch 31 de 848
Batch 32 de 848
Batch 33 de 848
Batch 34 de 848
Batch 35 de 848
Batch 36 de 848
Batch 37 de 848
Batch 38 de 848
Batch 39 de 848
Batch 40 de 848
Batch 41 de 848
Batch 42 de 848
Batch 43 de 848
Batch 44 de 848
Batch 45 de 848
Batch 46 de 848
Batch 47 de 848
Batch 48 de 848
Batch 49 de 848
Batch 50 de 848
Batch 51 de 848
Batch 52 de 848
Batch 53 de 848
Batch 54 de 848
Batch 55 de 848
Batch 56 de 848
Batch 57 de 848
Batch 58 de 848
Batch 59 de 848
Batch 60 de 848
Batch 61 de 848
Batch 62 de 848
Ba

##### Entrenamiento del modelo (30% de datos) v2

Utiliza:
- LR 1e-5
- weight_decay=0.01
- Dropout + Linear + Sigmoid
- Funcion de entrenamiento v2

In [ ]:
for epoch in range(EPOCHS):
  train_loss, train_accuracy = trainBert(epoch, model, training_loader, optimizer, device)

Epoch: 0, iteración: 0 de 2905, Loss: 0.06562252044677734
Accuracy: 0.625
Epoch: 0, iteración: 10 de 2905, Loss: 0.5719175666570664
Accuracy: 0.875
Epoch: 0, iteración: 20 de 2905, Loss: 0.5797299295663834
Accuracy: 0.625
Epoch: 0, iteración: 30 de 2905, Loss: 0.6012768596410751
Accuracy: 0.75
Epoch: 0, iteración: 40 de 2905, Loss: 0.5032866299152374
Accuracy: 0.875
Epoch: 0, iteración: 50 de 2905, Loss: 0.6381739705801011
Accuracy: 0.75
Epoch: 0, iteración: 60 de 2905, Loss: 0.5761262685060501
Accuracy: 0.625
Epoch: 0, iteración: 70 de 2905, Loss: 0.5888657093048095
Accuracy: 0.75
Epoch: 0, iteración: 80 de 2905, Loss: 0.6371831864118576
Accuracy: 0.75
Epoch: 0, iteración: 90 de 2905, Loss: 0.5663395345211029
Accuracy: 0.625
Epoch: 0, iteración: 100 de 2905, Loss: 0.6483917713165284
Accuracy: 0.875
Epoch: 0, iteración: 110 de 2905, Loss: 0.5219224393367767
Accuracy: 0.75
Epoch: 0, iteración: 120 de 2905, Loss: 0.6405450284481049
Accuracy: 0.875
Epoch: 0, iteración: 130 de 2905, Loss: 

In [ ]:
try:
    torch.save(model.state_dict(), "./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_30_v2.pth")
    print("Modelo guardado correctamente")
except Exception as e:
    print(f"Error al guardar el modelo: {e}")

Modelo guardado correctamente


##### Evaluación del modelo

In [ ]:
def validationBert(model, validation_loader, loss_fn, device):
    model.eval()
    sum_loss = 0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for i, data in enumerate(validation_loader, 0):
            ids = data['ids'].to(device)
            mask = data['mask'].to(device)
            token_type_ids = data['token_type_ids'].to(device)
            targets = data['target'].to(device)
            outputs = model(ids, mask, token_type_ids)

            # Calculate loss
            perdida = loss_fn(outputs.squeeze(), targets)
            sum_loss += perdida.item()

            # Calculate accuracy
            predictions = torch.sigmoid(outputs).cpu().numpy() >= 0.5
            correct_predictions += (predictions == targets.cpu().numpy()).sum()
            total_predictions += targets.size(0)

    # Calculate average loss and accuracy for the epoch
    epoch_loss = sum_loss / len(validation_loader)
    epoch_accuracy = correct_predictions / total_predictions

    # Append to history
    val_loss_history.append(epoch_loss)
    val_accuracy_history.append(epoch_accuracy)

    print(f"Accuracy Score = {epoch_accuracy}")

    return epoch_loss, epoch_accuracy

**Nota:** Esta celda no se encuentra ejecutada porque solo se usa cuando la evaluación de un modelo se ha hecho en una sesión distinta a su entrenamiento. Si se hace en la misma sesión, no tenemos necesidad de cargar el modelo (que acabamos de entrenar). Por el contrario, si lo entrenamos "hoy" y lo evaluamos "mañana", en vez de reentrenarlo para poder evaluarlo, solo es necesario cargarlo.

In [ ]:
if os.path.exists("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_30_v2.pth"):
  model = BERTClass()
  if device == "cuda":
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_30_v2.pth"))
    print("Modelo cargado correctamente")
  else:
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_30_v2.pth", map_location="cpu"))
    print("Modelo cargado correctamente con CPU")

  model.eval()
  model.to(device)

Modelo cargado correctamente


In [ ]:
loss_fn = torch.nn.BCEWithLogitsLoss()

for epoch in range(EPOCHS):
  avg_val_loss, val_accuracy = validationBert(model, testing_loader, loss_fn, device)

Batch 0 de 848
Batch 1 de 848
Batch 2 de 848
Batch 3 de 848
Batch 4 de 848
Batch 5 de 848
Batch 6 de 848
Batch 7 de 848
Batch 8 de 848
Batch 9 de 848
Batch 10 de 848
Batch 11 de 848
Batch 12 de 848
Batch 13 de 848
Batch 14 de 848
Batch 15 de 848
Batch 16 de 848
Batch 17 de 848
Batch 18 de 848
Batch 19 de 848
Batch 20 de 848
Batch 21 de 848
Batch 22 de 848
Batch 23 de 848
Batch 24 de 848
Batch 25 de 848
Batch 26 de 848
Batch 27 de 848
Batch 28 de 848
Batch 29 de 848
Batch 30 de 848
Batch 31 de 848
Batch 32 de 848
Batch 33 de 848
Batch 34 de 848
Batch 35 de 848
Batch 36 de 848
Batch 37 de 848
Batch 38 de 848
Batch 39 de 848
Batch 40 de 848
Batch 41 de 848
Batch 42 de 848
Batch 43 de 848
Batch 44 de 848
Batch 45 de 848
Batch 46 de 848
Batch 47 de 848
Batch 48 de 848
Batch 49 de 848
Batch 50 de 848
Batch 51 de 848
Batch 52 de 848
Batch 53 de 848
Batch 54 de 848
Batch 55 de 848
Batch 56 de 848
Batch 57 de 848
Batch 58 de 848
Batch 59 de 848
Batch 60 de 848
Batch 61 de 848
Batch 62 de 848
Ba

RuntimeError: Negation, the `-` operator, on a bool tensor is not supported. If you are trying to invert a mask, use the `~` or `logical_not()` operator instead.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

# Training and Validation Accuracy
plt.subplot(1, 2, 1)
plt.plot(train_accuracy_history, label='Training Accuracy')
plt.plot(val_accuracy_history, label='Validation Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)

# Training and Validation Loss
plt.subplot(1, 2, 2)
plt.plot(train_loss_history, label='Training Loss')
plt.plot(val_loss_history, label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)

plt.show()

##### Entrenamiento del modelo (50% de datos)

Utiliza:
- LR 1e-5
- weight_decay=0.01
- Dropout + Linear + Sigmoid

In [ ]:
for epoch in range(EPOCHS):
  trainBert(epoch)

Epoch: 0, iteración; 0 de 4841, Loss: 0.067620450258255
Epoch: 0, iteración; 10 de 4841, Loss: 0.6401959896087647
Epoch: 0, iteración; 20 de 4841, Loss: 0.6790625095367432
Epoch: 0, iteración; 30 de 4841, Loss: 0.636180305480957
Epoch: 0, iteración; 40 de 4841, Loss: 0.615803575515747
Epoch: 0, iteración; 50 de 4841, Loss: 0.5928953647613525
Epoch: 0, iteración; 60 de 4841, Loss: 0.5950827121734619
Epoch: 0, iteración; 70 de 4841, Loss: 0.6225556373596192
Epoch: 0, iteración; 80 de 4841, Loss: 0.6262527465820312
Epoch: 0, iteración; 90 de 4841, Loss: 0.6704085826873779
Epoch: 0, iteración; 100 de 4841, Loss: 0.5660880088806153
Epoch: 0, iteración; 110 de 4841, Loss: 0.5891512870788574
Epoch: 0, iteración; 120 de 4841, Loss: 0.5690562725067139
Epoch: 0, iteración; 130 de 4841, Loss: 0.6277326107025146
Epoch: 0, iteración; 140 de 4841, Loss: 0.5980307102203369
Epoch: 0, iteración; 150 de 4841, Loss: 0.5786302089691162
Epoch: 0, iteración; 160 de 4841, Loss: 0.6489881992340087
Epoch: 0, i

In [ ]:
try:
    torch.save(model.state_dict(), "./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_50.pth")
    print("Modelo guardado correctamente")
except Exception as e:
    print(f"Error al guardar el modelo: {e}")

Modelo guardado correctamente


##### Evaluación del modelo

In [ ]:
def validationBert(epoch):
  model.eval()
  fin_targets=[]
  fin_outputs=[]
  num_iteraciones = len(testing_loader)
  with torch.no_grad():
    for i, data in enumerate(testing_loader, 0):
      ids = data['ids'].to(device)
      mask = data['mask'].to(device)
      token_type_ids = data['token_type_ids'].to(device)
      targets = data['target'].to(device)

      print(f"Batch {i} de {num_iteraciones}")

      outputs = model(ids, mask, token_type_ids)
      fin_targets.extend(targets.cpu().detach().numpy().tolist())
      fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
  return fin_outputs, fin_targets

**Nota:** Esta celda no se encuentra ejecutada porque solo se usa cuando la evaluación de un modelo se ha hecho en una sesión distinta a su entrenamiento. Si se hace en la misma sesión, no tenemos necesidad de cargar el modelo (que acabamos de entrenar). Por el contrario, si lo entrenamos "hoy" y lo evaluamos "mañana", en vez de reentrenarlo para poder evaluarlo, solo es necesario cargarlo.

In [ ]:
if os.path.exists("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_50.pth"):
  print("La ruta existe")
  model = BERTClass()
  if device == "cuda":
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_50.pth"))
    print("Modelo cargado correctamente")
  else:
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_50.pth", map_location="cpu"))
    print("Modelo cargado correctamente con CPU")

  model.eval()
  model.to(device)

La ruta existe
Modelo cargado correctamente


In [ ]:
for epoch in range(EPOCHS):
  outputs, targets = validationBert(epoch)
  outputs = np.array(outputs) >= 0.5

  targets = np.array(targets).flatten().astype(int)

  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
  print(f"Accuracy Score = {accuracy}")
  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

Batch 0 de 606
Batch 1 de 606
Batch 2 de 606
Batch 3 de 606
Batch 4 de 606
Batch 5 de 606
Batch 6 de 606
Batch 7 de 606
Batch 8 de 606
Batch 9 de 606
Batch 10 de 606
Batch 11 de 606
Batch 12 de 606
Batch 13 de 606
Batch 14 de 606
Batch 15 de 606
Batch 16 de 606
Batch 17 de 606
Batch 18 de 606
Batch 19 de 606
Batch 20 de 606
Batch 21 de 606
Batch 22 de 606
Batch 23 de 606
Batch 24 de 606
Batch 25 de 606
Batch 26 de 606
Batch 27 de 606
Batch 28 de 606
Batch 29 de 606
Batch 30 de 606
Batch 31 de 606
Batch 32 de 606
Batch 33 de 606
Batch 34 de 606
Batch 35 de 606
Batch 36 de 606
Batch 37 de 606
Batch 38 de 606
Batch 39 de 606
Batch 40 de 606
Batch 41 de 606
Batch 42 de 606
Batch 43 de 606
Batch 44 de 606
Batch 45 de 606
Batch 46 de 606
Batch 47 de 606
Batch 48 de 606
Batch 49 de 606
Batch 50 de 606
Batch 51 de 606
Batch 52 de 606
Batch 53 de 606
Batch 54 de 606
Batch 55 de 606
Batch 56 de 606
Batch 57 de 606
Batch 58 de 606
Batch 59 de 606
Batch 60 de 606
Batch 61 de 606
Batch 62 de 606
Ba

##### Entrenamiento del modelo (50% de datos v2)

Utiliza:
- EPOCHS = 2
- LR 1e-3
- weight_decay=0.01
- Dropout + Linear + Sigmoid

In [ ]:
for epoch in range(EPOCHS):
  trainBert(epoch)

Epoch: 0, iteración; 0 de 4841, Loss: 0.05903231501579285
Epoch: 0, iteración; 10 de 4841, Loss: 0.5955514907836914
Epoch: 0, iteración; 20 de 4841, Loss: 0.5261722087860108
Epoch: 0, iteración; 30 de 4841, Loss: 0.5769295692443848
Epoch: 0, iteración; 40 de 4841, Loss: 0.5525286197662354
Epoch: 0, iteración; 50 de 4841, Loss: 0.7117277145385742
Epoch: 0, iteración; 60 de 4841, Loss: 0.5891118526458741
Epoch: 0, iteración; 70 de 4841, Loss: 0.5869298934936523
Epoch: 0, iteración; 80 de 4841, Loss: 0.6172382831573486
Epoch: 0, iteración; 90 de 4841, Loss: 0.5180586338043213
Epoch: 0, iteración; 100 de 4841, Loss: 0.5631719589233398
Epoch: 0, iteración; 110 de 4841, Loss: 0.5246448040008544
Epoch: 0, iteración; 120 de 4841, Loss: 0.620419979095459
Epoch: 0, iteración; 130 de 4841, Loss: 0.6034989356994629
Epoch: 0, iteración; 140 de 4841, Loss: 0.5394625663757324
Epoch: 0, iteración; 150 de 4841, Loss: 0.6000142097473145
Epoch: 0, iteración; 160 de 4841, Loss: 0.5840436935424804
Epoch: 0

In [ ]:
try:
    torch.save(model.state_dict(), "./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_50_v2.pth")
    print("Modelo guardado correctamente")
except Exception as e:
    print(f"Error al guardar el modelo: {e}")

Modelo guardado correctamente


##### Evaluación del modelo

In [ ]:
def validationBert(epoch):
  model.eval()
  fin_targets=[]
  fin_outputs=[]
  num_iteraciones = len(testing_loader)
  with torch.no_grad():
    for i, data in enumerate(testing_loader, 0):
      ids = data['ids'].to(device)
      mask = data['mask'].to(device)
      token_type_ids = data['token_type_ids'].to(device)
      targets = data['target'].to(device)

      print(f"Batch {i} de {num_iteraciones}")

      outputs = model(ids, mask, token_type_ids)
      fin_targets.extend(targets.cpu().detach().numpy().tolist())
      fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
  return fin_outputs, fin_targets

**Nota:** Esta celda no se encuentra ejecutada porque solo se usa cuando la evaluación de un modelo se ha hecho en una sesión distinta a su entrenamiento. Si se hace en la misma sesión, no tenemos necesidad de cargar el modelo (que acabamos de entrenar). Por el contrario, si lo entrenamos "hoy" y lo evaluamos "mañana", en vez de reentrenarlo para poder evaluarlo, solo es necesario cargarlo.

In [ ]:
if os.path.exists("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_50_v2.pth"):
  print("La ruta existe")
  model = BERTClass()
  if device == "cuda":
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_50_v2.pth"))
    print("Modelo cargado correctamente")
  else:
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_50_v2.pth", map_location="cpu"))
    print("Modelo cargado correctamente con CPU")

  model.eval()
  model.to(device)

La ruta existe
Modelo cargado correctamente


In [ ]:
for epoch in range(EPOCHS):
  outputs, targets = validationBert(epoch)
  outputs = np.array(outputs) >= 0.5
  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
  print(f"Accuracy Score = {accuracy}")
  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

Batch 0 de 606
Batch 1 de 606
Batch 2 de 606
Batch 3 de 606
Batch 4 de 606
Batch 5 de 606
Batch 6 de 606
Batch 7 de 606
Batch 8 de 606
Batch 9 de 606
Batch 10 de 606
Batch 11 de 606
Batch 12 de 606
Batch 13 de 606
Batch 14 de 606
Batch 15 de 606
Batch 16 de 606
Batch 17 de 606
Batch 18 de 606
Batch 19 de 606
Batch 20 de 606
Batch 21 de 606
Batch 22 de 606
Batch 23 de 606
Batch 24 de 606
Batch 25 de 606
Batch 26 de 606
Batch 27 de 606
Batch 28 de 606
Batch 29 de 606
Batch 30 de 606
Batch 31 de 606
Batch 32 de 606
Batch 33 de 606
Batch 34 de 606
Batch 35 de 606
Batch 36 de 606
Batch 37 de 606
Batch 38 de 606
Batch 39 de 606
Batch 40 de 606
Batch 41 de 606
Batch 42 de 606
Batch 43 de 606
Batch 44 de 606
Batch 45 de 606
Batch 46 de 606
Batch 47 de 606
Batch 48 de 606
Batch 49 de 606
Batch 50 de 606
Batch 51 de 606
Batch 52 de 606
Batch 53 de 606
Batch 54 de 606
Batch 55 de 606
Batch 56 de 606
Batch 57 de 606
Batch 58 de 606
Batch 59 de 606
Batch 60 de 606
Batch 61 de 606
Batch 62 de 606
Ba

##### Entrenamiento del modelo (80% de datos)

Utiliza:
- LR 1e-5
- weight_decay=0.01
- Dropout + Linear + Sigmoid

In [ ]:
for epoch in range(EPOCHS):
  trainBert(epoch)

Epoch: 0, iteración; 0 de 7745, Loss: 0.05297529101371765
Epoch: 0, iteración; 10 de 7745, Loss: 0.6039949417114258
Epoch: 0, iteración; 20 de 7745, Loss: 0.5772326946258545
Epoch: 0, iteración; 30 de 7745, Loss: 0.5921803951263428
Epoch: 0, iteración; 40 de 7745, Loss: 0.5428352355957031
Epoch: 0, iteración; 50 de 7745, Loss: 0.6100131034851074
Epoch: 0, iteración; 60 de 7745, Loss: 0.6429983139038086
Epoch: 0, iteración; 70 de 7745, Loss: 0.633080530166626
Epoch: 0, iteración; 80 de 7745, Loss: 0.6191935062408447
Epoch: 0, iteración; 90 de 7745, Loss: 0.5706310749053956
Epoch: 0, iteración; 100 de 7745, Loss: 0.5942049980163574
Epoch: 0, iteración; 110 de 7745, Loss: 0.5618804931640625
Epoch: 0, iteración; 120 de 7745, Loss: 0.6414105415344238
Epoch: 0, iteración; 130 de 7745, Loss: 0.5560259342193603
Epoch: 0, iteración; 140 de 7745, Loss: 0.5813461303710937
Epoch: 0, iteración; 150 de 7745, Loss: 0.5813619136810303
Epoch: 0, iteración; 160 de 7745, Loss: 0.6667360782623291
Epoch: 0

In [ ]:
try:
    torch.save(model.state_dict(), "./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_80.pth")
    print("Modelo guardado correctamente")
except Exception as e:
    print(f"Error al guardar el modelo: {e}")

Modelo guardado correctamente


##### Evaluación del modelo

In [ ]:
def validationBert(epoch):
  model.eval()
  fin_targets=[]
  fin_outputs=[]
  num_iteraciones = len(testing_loader)
  with torch.no_grad():
    for i, data in enumerate(testing_loader, 0):
      ids = data['ids'].to(device)
      mask = data['mask'].to(device)
      token_type_ids = data['token_type_ids'].to(device)
      targets = data['target'].to(device)

      print(f"Batch {i} de {num_iteraciones}")

      outputs = model(ids, mask, token_type_ids)
      fin_targets.extend(targets.cpu().detach().numpy().tolist())
      fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
  return fin_outputs, fin_targets

**Nota:** Esta celda no se encuentra ejecutada porque solo se usa cuando la evaluación de un modelo se ha hecho en una sesión distinta a su entrenamiento. Si se hace en la misma sesión, no tenemos necesidad de cargar el modelo (que acabamos de entrenar). Por el contrario, si lo entrenamos "hoy" y lo evaluamos "mañana", en vez de reentrenarlo para poder evaluarlo, solo es necesario cargarlo.

In [ ]:
if os.path.exists("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_80.pth"):
  model = BERTClass()
  if device == "cuda":
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_80.pth"))
    print("Modelo cargado correctamente")
  else:
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_80.pth", map_location="cpu"))
    print("Modelo cargado correctamente con CPU")

  model.eval()
  model.to(device)

Modelo cargado correctamente


In [ ]:
for epoch in range(EPOCHS):
  outputs, targets = validationBert(epoch)
  outputs = np.array(outputs) >= 0.5

  targets = np.array(targets).flatten().astype(int)

  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
  print(f"Accuracy Score = {accuracy}")
  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

Batch 0 de 243
Batch 1 de 243
Batch 2 de 243
Batch 3 de 243
Batch 4 de 243
Batch 5 de 243
Batch 6 de 243
Batch 7 de 243
Batch 8 de 243
Batch 9 de 243
Batch 10 de 243
Batch 11 de 243
Batch 12 de 243
Batch 13 de 243
Batch 14 de 243
Batch 15 de 243
Batch 16 de 243
Batch 17 de 243
Batch 18 de 243
Batch 19 de 243
Batch 20 de 243
Batch 21 de 243
Batch 22 de 243
Batch 23 de 243
Batch 24 de 243
Batch 25 de 243
Batch 26 de 243
Batch 27 de 243
Batch 28 de 243
Batch 29 de 243
Batch 30 de 243
Batch 31 de 243
Batch 32 de 243
Batch 33 de 243
Batch 34 de 243
Batch 35 de 243
Batch 36 de 243
Batch 37 de 243
Batch 38 de 243
Batch 39 de 243
Batch 40 de 243
Batch 41 de 243
Batch 42 de 243
Batch 43 de 243
Batch 44 de 243
Batch 45 de 243
Batch 46 de 243
Batch 47 de 243
Batch 48 de 243
Batch 49 de 243
Batch 50 de 243
Batch 51 de 243
Batch 52 de 243
Batch 53 de 243
Batch 54 de 243
Batch 55 de 243
Batch 56 de 243
Batch 57 de 243
Batch 58 de 243
Batch 59 de 243
Batch 60 de 243
Batch 61 de 243
Batch 62 de 243
Ba

####2.4.4.3 Modelo 2: RoBERTa

##### Generación Dataset y Dataloader

In [ ]:
training_loader, testing_loader = generate_loaders(datos, tokenizerRB,
                                                   restaurantsDataset,
                                                   DataCollatorRestaurant
                                                   )

FULL Dataset:(77448, 4)
TRAIN Dataset: (61958, 4)
TEST Dataset: (15490, 4)


##### Definición del modelo

In [ ]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.dropout = 0.2
        self.hidden_embd = 768
        self.output_layer = 1

        # Layers
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        #self.l2 = torch.nn.Linear(self.hidden_embd, 256)
        #self.l3 = torch.nn.Linear(256, 64)
        self.l4 = torch.nn.Dropout(self.dropout)
        # self.l5 = torch.nn.Linear(64, self.output_layer)
        self.l5 = torch.nn.Linear(self.hidden_embd, self.output_layer)
        self.l6 = torch.nn.Sigmoid()

    def forward(self, ids, mask, token_type_ids):
        _, output_1 = self.l1(ids, attention_mask=mask, token_type_ids=token_type_ids, return_dict=False)
        #output_2 = self.l2(output_1)
        #output_3 = self.l3(output_2)
        output_4 = self.l4(output_1)
        output = self.l5(output_4)
        output = self.l6(output)
        return output

model = RobertaClass()
model.to(device)

RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE,
                             weight_decay=0.01)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 1e-05
    maximize: False
    weight_decay: 0.01
)

##### Función de entrenamiento

In [ ]:
def trainRoberta(epoch):
  model.train()
  loss_fn = torch.nn.BCELoss() #funcion perdida categorical cross entropy
  num_iteraciones = len(training_loader)
  sum_loss = 0

  for iteracion,data in enumerate(training_loader, 0):
    ids = data['ids'].to(device)
    mask = data['mask'].to(device)
    targets = data['target'].to(device)
    token_type_ids = data["token_type_ids"].to(device)

    output = model(ids, mask, token_type_ids)
    optimizer.zero_grad()

    perdida = loss_fn(output.squeeze(), targets)
    with torch.no_grad():
      sum_loss+=perdida
      if iteracion % PASOS_POR_INTERVALO == 0:
        print(f'Epoch: {epoch}, iteración; {iteracion:6} de {num_iteraciones}, Loss: {sum_loss.cpu().numpy()/PASOS_POR_INTERVALO}')
        sum_loss = 0

    optimizer.zero_grad()
    perdida.backward()
    optimizer.step()

##### Entrenamiento del modelo (30% de datos)

Utiliza:
- LR 1e-5
- weight_decay=0.01
- Dropout + Linear + Sigmoid

In [ ]:
for epoch in range(EPOCHS):
  trainRoberta(epoch)

Epoch: 0, iteración;      0 de 2905, Loss: 0.06860789060592651
Epoch: 0, iteración;     10 de 2905, Loss: 0.6764469623565674
Epoch: 0, iteración;     20 de 2905, Loss: 0.648057222366333
Epoch: 0, iteración;     30 de 2905, Loss: 0.604004955291748
Epoch: 0, iteración;     40 de 2905, Loss: 0.6138870716094971
Epoch: 0, iteración;     50 de 2905, Loss: 0.68815598487854
Epoch: 0, iteración;     60 de 2905, Loss: 0.6525362491607666
Epoch: 0, iteración;     70 de 2905, Loss: 0.625678300857544
Epoch: 0, iteración;     80 de 2905, Loss: 0.5612018585205079
Epoch: 0, iteración;     90 de 2905, Loss: 0.5164202213287353
Epoch: 0, iteración;    100 de 2905, Loss: 0.5604738712310791
Epoch: 0, iteración;    110 de 2905, Loss: 0.5884951114654541
Epoch: 0, iteración;    120 de 2905, Loss: 0.5894658088684082
Epoch: 0, iteración;    130 de 2905, Loss: 0.6428716659545899
Epoch: 0, iteración;    140 de 2905, Loss: 0.5927660465240479
Epoch: 0, iteración;    150 de 2905, Loss: 0.5822919845581055
Epoch: 0, it

In [ ]:
try:
    torch.save(model.state_dict(), "./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_roberta_30.pth")
    print("Modelo guardado correctamente")
except Exception as e:
    print(f"Error al guardar el modelo: {e}")

Modelo guardado correctamente


##### Evaluación del modelo

In [ ]:
def validationRoberta(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    num_iteraciones = len(testing_loader)
    with torch.no_grad():
        for i, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device)
            mask = data['mask'].to(device)
            token_type_ids = data['token_type_ids'].to(device)
            targets = data['target'].to(device)

            print(f"Iteración: {i:6} de {num_iteraciones}")

            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

**Nota:** Esta celda no se encuentra ejecutada porque solo se usa cuando la evaluación de un modelo se ha hecho en una sesión distinta a su entrenamiento. Si se hace en la misma sesión, no tenemos necesidad de cargar el modelo (que acabamos de entrenar). Por el contrario, si lo entrenamos "hoy" y lo evaluamos "mañana", en vez de reentrenarlo para poder evaluarlo, solo es necesario cargarlo.

In [ ]:
if os.path.exists("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_roberta_30.pth"):
  model = RobertaClass()
  if device == "cuda":
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_roberta_30.pth"))
    print("Modelo cargado correctamente")
  else:
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_roberta_30.pth", map_location="cpu"))
    print("Modelo cargado correctamente con CPU")

  model.eval()
  model.to(device)

In [ ]:
for epoch in range(EPOCHS):
  outputs, targets = validationRoberta(epoch)
  outputs = np.array(outputs) >= 0.5
  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
  print(f"Accuracy Score = {accuracy}")
  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

Iteración:      0 de 848
Iteración:      1 de 848
Iteración:      2 de 848
Iteración:      3 de 848
Iteración:      4 de 848
Iteración:      5 de 848
Iteración:      6 de 848
Iteración:      7 de 848
Iteración:      8 de 848
Iteración:      9 de 848
Iteración:     10 de 848
Iteración:     11 de 848
Iteración:     12 de 848
Iteración:     13 de 848
Iteración:     14 de 848
Iteración:     15 de 848
Iteración:     16 de 848
Iteración:     17 de 848
Iteración:     18 de 848
Iteración:     19 de 848
Iteración:     20 de 848
Iteración:     21 de 848
Iteración:     22 de 848
Iteración:     23 de 848
Iteración:     24 de 848
Iteración:     25 de 848
Iteración:     26 de 848
Iteración:     27 de 848
Iteración:     28 de 848
Iteración:     29 de 848
Iteración:     30 de 848
Iteración:     31 de 848
Iteración:     32 de 848
Iteración:     33 de 848
Iteración:     34 de 848
Iteración:     35 de 848
Iteración:     36 de 848
Iteración:     37 de 848
Iteración:     38 de 848
Iteración:     39 de 848


##### Entrenamiento del modelo (50% de datos)

Utiliza:
- LR 1e-5
- weight_decay=0.01
- Dropout + Linear + Sigmoid

In [ ]:
for epoch in range(EPOCHS):
  trainRoberta(epoch)

Epoch: 0, iteración;      0 de 4841, Loss: 0.0690148115158081
Epoch: 0, iteración;     10 de 4841, Loss: 0.6695529460906983
Epoch: 0, iteración;     20 de 4841, Loss: 0.6651931285858155
Epoch: 0, iteración;     30 de 4841, Loss: 0.6305753707885742
Epoch: 0, iteración;     40 de 4841, Loss: 0.6418673038482666
Epoch: 0, iteración;     50 de 4841, Loss: 0.667976188659668
Epoch: 0, iteración;     60 de 4841, Loss: 0.6516317367553711
Epoch: 0, iteración;     70 de 4841, Loss: 0.6127129554748535
Epoch: 0, iteración;     80 de 4841, Loss: 0.6398096084594727
Epoch: 0, iteración;     90 de 4841, Loss: 0.6491200447082519
Epoch: 0, iteración;    100 de 4841, Loss: 0.6119801998138428
Epoch: 0, iteración;    110 de 4841, Loss: 0.5749374866485596
Epoch: 0, iteración;    120 de 4841, Loss: 0.5183465480804443
Epoch: 0, iteración;    130 de 4841, Loss: 0.5098889350891114
Epoch: 0, iteración;    140 de 4841, Loss: 0.5941874504089355
Epoch: 0, iteración;    150 de 4841, Loss: 0.4488194465637207
Epoch: 0,

In [ ]:
try:
    torch.save(model.state_dict(), "./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_roberta_50.pth")
    print("Modelo guardado correctamente")
except Exception as e:
    print(f"Error al guardar el modelo: {e}")

Modelo guardado correctamente


##### Evaluación del modelo

In [ ]:
def validationRoberta(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    num_iteraciones = len(testing_loader)
    with torch.no_grad():
        for i, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device)
            mask = data['mask'].to(device)
            token_type_ids = data['token_type_ids'].to(device)
            targets = data['target'].to(device)

            print(f"Iteración: {i:6} de {num_iteraciones}")

            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

**Nota:** Esta celda no se encuentra ejecutada porque solo se usa cuando la evaluación de un modelo se ha hecho en una sesión distinta a su entrenamiento. Si se hace en la misma sesión, no tenemos necesidad de cargar el modelo (que acabamos de entrenar). Por el contrario, si lo entrenamos "hoy" y lo evaluamos "mañana", en vez de reentrenarlo para poder evaluarlo, solo es necesario cargarlo.

In [ ]:
if os.path.exists("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_roberta_50.pth"):
  model = RobertaClass()
  if device == "cuda":
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_roberta_50.pth"))
    print("Modelo cargado correctamente")
  else:
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_roberta_50.pth", map_location="cpu"))
    print("Modelo cargado correctamente con CPU")

  model.eval()
  model.to(device)

In [ ]:
for epoch in range(EPOCHS):
  outputs, targets = validationRoberta(epoch)
  outputs = np.array(outputs) >= 0.5
  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
  print(f"Accuracy Score = {accuracy}")
  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

Iteración:      0 de 606
Iteración:      1 de 606
Iteración:      2 de 606
Iteración:      3 de 606
Iteración:      4 de 606
Iteración:      5 de 606
Iteración:      6 de 606
Iteración:      7 de 606
Iteración:      8 de 606
Iteración:      9 de 606
Iteración:     10 de 606
Iteración:     11 de 606
Iteración:     12 de 606
Iteración:     13 de 606
Iteración:     14 de 606
Iteración:     15 de 606
Iteración:     16 de 606
Iteración:     17 de 606
Iteración:     18 de 606
Iteración:     19 de 606
Iteración:     20 de 606
Iteración:     21 de 606
Iteración:     22 de 606
Iteración:     23 de 606
Iteración:     24 de 606
Iteración:     25 de 606
Iteración:     26 de 606
Iteración:     27 de 606
Iteración:     28 de 606
Iteración:     29 de 606
Iteración:     30 de 606
Iteración:     31 de 606
Iteración:     32 de 606
Iteración:     33 de 606
Iteración:     34 de 606
Iteración:     35 de 606
Iteración:     36 de 606
Iteración:     37 de 606
Iteración:     38 de 606
Iteración:     39 de 606


##### Entrenamiento del modelo (80% de datos)

Utiliza:
- LR 1e-5
- weight_decay=0.01
- Dropout + Linear + Sigmoid

In [ ]:
for epoch in range(EPOCHS):
  trainRoberta(epoch)

Epoch: 0, iteración;      0 de 7745, Loss: 0.07203119993209839
Epoch: 0, iteración;     10 de 7745, Loss: 0.6863770961761475
Epoch: 0, iteración;     20 de 7745, Loss: 0.6773520946502686
Epoch: 0, iteración;     30 de 7745, Loss: 0.6759041309356689
Epoch: 0, iteración;     40 de 7745, Loss: 0.6124179363250732
Epoch: 0, iteración;     50 de 7745, Loss: 0.6373193740844727
Epoch: 0, iteración;     60 de 7745, Loss: 0.5993459701538086
Epoch: 0, iteración;     70 de 7745, Loss: 0.5959677219390869
Epoch: 0, iteración;     80 de 7745, Loss: 0.5703641414642334
Epoch: 0, iteración;     90 de 7745, Loss: 0.5545778274536133
Epoch: 0, iteración;    100 de 7745, Loss: 0.5832770347595215
Epoch: 0, iteración;    110 de 7745, Loss: 0.5973462104797364
Epoch: 0, iteración;    120 de 7745, Loss: 0.6187351226806641
Epoch: 0, iteración;    130 de 7745, Loss: 0.5685255050659179
Epoch: 0, iteración;    140 de 7745, Loss: 0.5912712574005127
Epoch: 0, iteración;    150 de 7745, Loss: 0.5497786521911621
Epoch: 

In [ ]:
try:
    torch.save(model.state_dict(), "./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_roberta_80.pth")
    print("Modelo guardado correctamente")
except Exception as e:
    print(f"Error al guardar el modelo: {e}")

Modelo guardado correctamente


##### Evaluación del modelo

In [ ]:
def validationRoberta(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    num_iteraciones = len(testing_loader)
    with torch.no_grad():
        for i, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device)
            mask = data['mask'].to(device)
            token_type_ids = data['token_type_ids'].to(device)
            targets = data['target'].to(device)

            print(f"Iteración: {i:6} de {num_iteraciones}")

            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

**Nota:** Esta celda no se encuentra ejecutada porque solo se usa cuando la evaluación de un modelo se ha hecho en una sesión distinta a su entrenamiento. Si se hace en la misma sesión, no tenemos necesidad de cargar el modelo (que acabamos de entrenar). Por el contrario, si lo entrenamos "hoy" y lo evaluamos "mañana", en vez de reentrenarlo para poder evaluarlo, solo es necesario cargarlo.

In [ ]:
if os.path.exists("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_roberta_80.pth"):
  model = RobertaClass()
  if device == "cuda":
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_roberta_80.pth"))
    print("Modelo cargado correctamente")
  else:
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_roberta_80.pth", map_location="cpu"))
    print("Modelo cargado correctamente con CPU")

  model.eval()
  model.to(device)

In [ ]:
for epoch in range(EPOCHS):
  outputs, targets = validationRoberta(epoch)
  outputs = np.array(outputs) >= 0.5
  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
  print(f"Accuracy Score = {accuracy}")
  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

Iteración:      0 de 243
Iteración:      1 de 243
Iteración:      2 de 243
Iteración:      3 de 243
Iteración:      4 de 243
Iteración:      5 de 243
Iteración:      6 de 243
Iteración:      7 de 243
Iteración:      8 de 243
Iteración:      9 de 243
Iteración:     10 de 243
Iteración:     11 de 243
Iteración:     12 de 243
Iteración:     13 de 243
Iteración:     14 de 243
Iteración:     15 de 243
Iteración:     16 de 243
Iteración:     17 de 243
Iteración:     18 de 243
Iteración:     19 de 243
Iteración:     20 de 243
Iteración:     21 de 243
Iteración:     22 de 243
Iteración:     23 de 243
Iteración:     24 de 243
Iteración:     25 de 243
Iteración:     26 de 243
Iteración:     27 de 243
Iteración:     28 de 243
Iteración:     29 de 243
Iteración:     30 de 243
Iteración:     31 de 243
Iteración:     32 de 243
Iteración:     33 de 243
Iteración:     34 de 243
Iteración:     35 de 243
Iteración:     36 de 243
Iteración:     37 de 243
Iteración:     38 de 243
Iteración:     39 de 243


##2.5 Aproximación 2: Separando las dos reviews de cada restaurante


En esta aproximación, se volverán a realizar los pasos de la sección 2.4, pero esta vez, haciendo uso de una cantidad considerablemente mayor del conjunto de los datos.

###2.5.1 Carga de datos

En este paso, se cargan los datos preprocesados de la sección 1.2 Datos divididos.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
datos = pd.read_feather("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/valoraciones_Separados_train.feather")
datos

,Reviews,Sentimiento
0,A most friendly welcome,1
1,Nice treat,0
2,"Lovely interior design', ""Don't let the lack o...",1
3,Nice Food and Good Service,1
4,Just ok,0
...,...,...
142763,Great Chinese food,0
142764,Take a detour off the main drag,1
142765,Good Burrito,1
142766,Super Rude,0


###2.5.2 Definición tokenizadores y modelos

In [ ]:
tokenizerB = BertTokenizer.from_pretrained("bert-base-uncased")
modelB = BertModel.from_pretrained("bert-base-uncased")
configB = AutoConfig.from_pretrained("bert-base-uncased")

In [ ]:
tokenizerRB = RobertaTokenizer.from_pretrained("roberta-base")
modelRB = RobertaModel.from_pretrained("roberta-base")
configRB = AutoConfig.from_pretrained("roberta-base")

###2.5.3 Función de generación de conjuntos de train y test: DataLoaders

In [ ]:
def generate_loaders(dataframe, tokenizer, dataset, collator):
  train_dataset = dataframe.sample(frac=TRAIN_SIZE, random_state=0)
  test_dataset = dataframe.drop(train_dataset.index).reset_index(drop=True)
  train_dataset = train_dataset.reset_index(drop=True)

  print(f"FULL Dataset:{dataframe.shape}")
  print(f"TRAIN Dataset: {train_dataset.shape}")
  print(f"TEST Dataset: {test_dataset.shape}")

  training_set = dataset(train_dataset, MAX_LEN)
  testing_set = dataset(test_dataset, MAX_LEN)
  dc = collator(tokenizer, MAX_LEN)


  train_params = {'batch_size': TRAIN_BATCH_SIZE,
                  'shuffle': True,
                  'num_workers': 0,
                  'collate_fn': dc
                  }

  test_params = { 'batch_size': VALID_BATCH_SIZE,
                  'shuffle': True,
                  'num_workers': 0,
                  'collate_fn': dc
                }

  training_loader = DataLoader(training_set, **train_params)
  testing_loader = DataLoader(testing_set, **test_params)
  return training_loader, testing_loader

###2.5.4 Definición restaurantsDataset y DataCollatorRestaurant

Esta clase es la encargada de elaborar las muestras con las que van a entrenar los modelos.

Cada muestra consiste en:
- La etiqueta *sentimiento*, que representa el valor de la columna `Sentimineto` de los datos, es decir, si el sentimiento es positivo o negativo.
- `Reviews`, comentarios que se utilizan para entrenar el modelo.

En esta aproximación se busca generar el *embedding* de las reseñas sobre cada *review*.

In [ ]:
type(datos)

pandas.core.frame.DataFrame

In [ ]:
datos

,Reviews,Sentimiento
0,A most friendly welcome,1
1,Nice treat,0
2,"Lovely interior design', ""Don't let the lack o...",1
3,Nice Food and Good Service,1
4,Just ok,0
...,...,...
142763,Great Chinese food,0
142764,Take a detour off the main drag,1
142765,Good Burrito,1
142766,Super Rude,0


In [ ]:
class restaurantsDataset(Dataset):
  def __init__(self, dataframe, max_len):
    self.data = dataframe
    self.comentario = dataframe.Reviews
    self.targets = self.data.Sentimiento
    self.max_len = max_len

  def __len__(self):
    return len(self.comentario)

  def __getitem__(self, index):
    comment = self.comentario[index]
    target = self.targets[index]

    return {
        "comentario": comment,
        "target": target
    }

In [ ]:
datasetRestaurants = restaurantsDataset(datos, MAX_LEN)
salida = datasetRestaurants.__getitem__(0)
salida

{'comentario': 'A most friendly welcome', 'target': 1}

In [ ]:
aux = encoder_input = tokenizerB(
          salida["comentario"],
          None,
          add_special_tokens=True,
          truncation=True,
          max_length=MAX_LEN,
          pad_to_max_length=True,
          return_tensors = "pt",
          return_token_type_ids=True
          )
aux["input_ids"]

tensor([[ 101, 1037, 2087, 5379, 6160,  102,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,  

In [ ]:
class DataCollatorRestaurant:
    def __init__(self, tokenizer, max_len=512):
      self.tokenizer = tokenizer
      self.max_len = max_len

    def __call__(self, input_batch):
      data_frame = pd.DataFrame(input_batch)
      batch_dict = {column: data_frame[column].tolist() for column in data_frame}

      encoder_input = self.tokenizer(
          batch_dict["comentario"],
          None,
          add_special_tokens=True,
          truncation=True,
          max_length=self.max_len,
          pad_to_max_length=True,
          return_tensors = "pt",
          return_token_type_ids=True
          )

      return {
      'ids': encoder_input["input_ids"],
      'mask': encoder_input["attention_mask"],
      "target": torch.Tensor(batch_dict["target"]),
      "token_type_ids": encoder_input["token_type_ids"]
    }

###2.5.5 Modelo 1: BERT

##### Definición Dataset y Dataloader

Empleando la función anteriormente desarrollada, podemos generar los *loaders* sin tener que especificar nada más que el *tokenizer* concreto y el *Dataset* y el *DataCollator*.

In [ ]:
training_loader, testing_loader = generate_loaders(datos,
                                                   tokenizerB,
                                                   restaurantsDataset,
                                                   DataCollatorRestaurant
                                                   )

FULL Dataset:(142768, 2)
TRAIN Dataset: (114214, 2)
TEST Dataset: (28554, 2)


##### Definición del modelo

In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.dropout = 0.2
        self.hidden_embd = 768
        self.output_layer = 1

        # Layers
        self.l1 = BertModel.from_pretrained('bert-base-uncased')
        #self.l2 = torch.nn.Linear(self.hidden_embd, 256)
        #self.l3 = torch.nn.Linear(256, 64)
        self.l4 = torch.nn.Dropout(self.dropout)
        # self.l5 = torch.nn.Linear(64, self.output_layer)
        self.l5 = torch.nn.Linear(self.hidden_embd, self.output_layer)
        self.l6 = torch.nn.Sigmoid()

    def forward(self, ids, mask, token_type_ids):
        _, output_1 = self.l1(ids, attention_mask=mask, token_type_ids=token_type_ids, return_dict=False)
        #output_2 = self.l2(output_1)
        #output_3 = self.l3(output_2)
        output_4 = self.l4(output_1)
        output_5 = self.l5(output_4)
        output = self.l6(output_5)
        return output

model = BERTClass()
model.to(device)

BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE,
                             weight_decay=0.01)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0.01
)

##### Función de entrenamiento


In [ ]:
def trainBert(epoch):
  model.train()
  loss_fn = torch.nn.BCEWithLogitsLoss()
  num_iteraciones = len(training_loader)
  sum_loss = 0

  for iteracion, data in enumerate(training_loader, 0):
    ids = data['ids'].to(device)
    mask = data['mask'].to(device)
    targets = data['target'].to(device)
    token_type_ids = data["token_type_ids"].to(device)

    output = model(ids, mask, token_type_ids)
    optimizer.zero_grad()

    perdida = loss_fn(output.squeeze(), targets)
    with torch.no_grad():
      sum_loss+=perdida
      if iteracion % PASOS_POR_INTERVALO ==0:
        print(f'Epoch: {epoch}, iteración; {iteracion} de {num_iteraciones}, Loss: {sum_loss.cpu().numpy()/PASOS_POR_INTERVALO}')
        sum_loss = 0

    optimizer.zero_grad()
    perdida.backward()
    optimizer.step()

##### Entrenamiento del modelo (30% de datos)

Utiliza:
- LR 1e-5
- weight_decay=0.01
- Dropout + Linear + Sigmoid

In [ ]:
for epoch in range(EPOCHS):
  trainBert(epoch)

Epoch: 0, iteración; 0 de 5354, Loss: 0.06584186553955078
Epoch: 0, iteración; 10 de 5354, Loss: 0.5883976936340332
Epoch: 0, iteración; 20 de 5354, Loss: 0.632751178741455
Epoch: 0, iteración; 30 de 5354, Loss: 0.6096447944641114
Epoch: 0, iteración; 40 de 5354, Loss: 0.5748745918273925
Epoch: 0, iteración; 50 de 5354, Loss: 0.5510220050811767
Epoch: 0, iteración; 60 de 5354, Loss: 0.617931318283081
Epoch: 0, iteración; 70 de 5354, Loss: 0.5796681404113769
Epoch: 0, iteración; 80 de 5354, Loss: 0.5210927963256836
Epoch: 0, iteración; 90 de 5354, Loss: 0.6179694175720215
Epoch: 0, iteración; 100 de 5354, Loss: 0.6450166702270508
Epoch: 0, iteración; 110 de 5354, Loss: 0.5910777091979981
Epoch: 0, iteración; 120 de 5354, Loss: 0.5932302474975586
Epoch: 0, iteración; 130 de 5354, Loss: 0.5541589260101318
Epoch: 0, iteración; 140 de 5354, Loss: 0.5372121810913086
Epoch: 0, iteración; 150 de 5354, Loss: 0.646149730682373
Epoch: 0, iteración; 160 de 5354, Loss: 0.6178732872009277
Epoch: 0, 

In [ ]:
try:
    torch.save(model.state_dict(), "./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_30_separados.pth")
    print("Modelo guardado correctamente")
except Exception as e:
    print(f"Error al guardar el modelo: {e}")

Modelo guardado correctamente


##### Evaluación del modelo

In [ ]:
def validationBert(epoch):
  model.eval()
  fin_targets=[]
  fin_outputs=[]
  num_iteraciones = len(testing_loader)
  with torch.no_grad():
    for i, data in enumerate(testing_loader, 0):
      ids = data['ids'].to(device)
      mask = data['mask'].to(device)
      token_type_ids = data['token_type_ids'].to(device)
      targets = data['target'].to(device)

      print(f"Batch {i} de {num_iteraciones}")

      outputs = model(ids, mask, token_type_ids)
      fin_targets.extend(targets.cpu().detach().numpy().tolist())
      fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
  return fin_outputs, fin_targets

**Nota:** Esta celda no se encuentra ejecutada porque solo se usa cuando la evaluación de un modelo se ha hecho en una sesión distinta a su entrenamiento. Si se hace en la misma sesión, no tenemos necesidad de cargar el modelo (que acabamos de entrenar). Por el contrario, si lo entrenamos "hoy" y lo evaluamos "mañana", en vez de reentrenarlo para poder evaluarlo, solo es necesario cargarlo.

In [ ]:
if os.path.exists("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_30_separados.pth"):
  model = BERTClass()
  if device == "cuda":
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_30_separados.pth"))
    print("Modelo cargado correctamente")
  else:
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_30_separados.pth", map_location="cpu"))
    print("Modelo cargado correctamente con CPU")

  model.eval()
  model.to(device)

Modelo cargado correctamente


In [ ]:
for epoch in range(EPOCHS):
  outputs, targets = validationBert(epoch)
  outputs = np.array(outputs) >= 0.5

  targets = np.array(targets).flatten().astype(int)

  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
  print(f"Accuracy Score = {accuracy}")
  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

Batch 0 de 1562
Batch 1 de 1562
Batch 2 de 1562
Batch 3 de 1562
Batch 4 de 1562
Batch 5 de 1562
Batch 6 de 1562
Batch 7 de 1562
Batch 8 de 1562
Batch 9 de 1562
Batch 10 de 1562
Batch 11 de 1562
Batch 12 de 1562
Batch 13 de 1562
Batch 14 de 1562
Batch 15 de 1562
Batch 16 de 1562
Batch 17 de 1562
Batch 18 de 1562
Batch 19 de 1562
Batch 20 de 1562
Batch 21 de 1562
Batch 22 de 1562
Batch 23 de 1562
Batch 24 de 1562
Batch 25 de 1562
Batch 26 de 1562
Batch 27 de 1562
Batch 28 de 1562
Batch 29 de 1562
Batch 30 de 1562
Batch 31 de 1562
Batch 32 de 1562
Batch 33 de 1562
Batch 34 de 1562
Batch 35 de 1562
Batch 36 de 1562
Batch 37 de 1562
Batch 38 de 1562
Batch 39 de 1562
Batch 40 de 1562
Batch 41 de 1562
Batch 42 de 1562
Batch 43 de 1562
Batch 44 de 1562
Batch 45 de 1562
Batch 46 de 1562
Batch 47 de 1562
Batch 48 de 1562
Batch 49 de 1562
Batch 50 de 1562
Batch 51 de 1562
Batch 52 de 1562
Batch 53 de 1562
Batch 54 de 1562
Batch 55 de 1562
Batch 56 de 1562
Batch 57 de 1562
Batch 58 de 1562
Batch 5

##### Entrenamiento del modelo (50% de datos)

Utiliza:
- LR 1e-5
- weight_decay=0.01
- Dropout + Linear + Sigmoid

In [ ]:
for epoch in range(EPOCHS):
  trainBert(epoch)

Epoch: 0, iteración; 0 de 8923, Loss: 0.06716524362564087
Epoch: 0, iteración; 10 de 8923, Loss: 0.5760684490203858
Epoch: 0, iteración; 20 de 8923, Loss: 0.5990167617797851
Epoch: 0, iteración; 30 de 8923, Loss: 0.6765484809875488
Epoch: 0, iteración; 40 de 8923, Loss: 0.5990793704986572
Epoch: 0, iteración; 50 de 8923, Loss: 0.5697183609008789
Epoch: 0, iteración; 60 de 8923, Loss: 0.5550312995910645
Epoch: 0, iteración; 70 de 8923, Loss: 0.5230428218841553
Epoch: 0, iteración; 80 de 8923, Loss: 0.5828387260437011
Epoch: 0, iteración; 90 de 8923, Loss: 0.6101259708404541
Epoch: 0, iteración; 100 de 8923, Loss: 0.5997524738311768
Epoch: 0, iteración; 110 de 8923, Loss: 0.5246726989746093
Epoch: 0, iteración; 120 de 8923, Loss: 0.6086777210235595
Epoch: 0, iteración; 130 de 8923, Loss: 0.582326602935791
Epoch: 0, iteración; 140 de 8923, Loss: 0.5876935482025146
Epoch: 0, iteración; 150 de 8923, Loss: 0.5486178398132324
Epoch: 0, iteración; 160 de 8923, Loss: 0.5636777877807617
Epoch: 0

In [ ]:
try:
    torch.save(model.state_dict(), "./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_50_separados.pth")
    print("Modelo guardado correctamente")
except Exception as e:
    print(f"Error al guardar el modelo: {e}")

Modelo guardado correctamente


##### Evaluación del modelo

In [ ]:
def validationBert(epoch):
  model.eval()
  fin_targets=[]
  fin_outputs=[]
  num_iteraciones = len(testing_loader)
  with torch.no_grad():
    for i, data in enumerate(testing_loader, 0):
      ids = data['ids'].to(device)
      mask = data['mask'].to(device)
      token_type_ids = data['token_type_ids'].to(device)
      targets = data['target'].to(device)

      print(f"Batch {i} de {num_iteraciones}")

      outputs = model(ids, mask, token_type_ids)
      fin_targets.extend(targets.cpu().detach().numpy().tolist())
      fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
  return fin_outputs, fin_targets

**Nota:** Esta celda no se encuentra ejecutada porque solo se usa cuando la evaluación de un modelo se ha hecho en una sesión distinta a su entrenamiento. Si se hace en la misma sesión, no tenemos necesidad de cargar el modelo (que acabamos de entrenar). Por el contrario, si lo entrenamos "hoy" y lo evaluamos "mañana", en vez de reentrenarlo para poder evaluarlo, solo es necesario cargarlo.

In [ ]:
if os.path.exists("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_50_separados.pth"):
  print("La ruta existe")
  model = BERTClass()
  if device == "cuda":
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_50_separados.pth"))
    print("Modelo cargado correctamente")
  else:
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_50_separados.pth", map_location="cpu"))
    print("Modelo cargado correctamente con CPU")

  model.eval()
  model.to(device)

La ruta existe
Modelo cargado correctamente


In [ ]:
for epoch in range(EPOCHS):
  outputs, targets = validationBert(epoch)
  outputs = np.array(outputs) >= 0.5

  targets = np.array(targets).flatten().astype(int)

  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
  print(f"Accuracy Score = {accuracy}")
  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

Batch 0 de 1116
Batch 1 de 1116
Batch 2 de 1116
Batch 3 de 1116
Batch 4 de 1116
Batch 5 de 1116
Batch 6 de 1116
Batch 7 de 1116
Batch 8 de 1116
Batch 9 de 1116
Batch 10 de 1116
Batch 11 de 1116
Batch 12 de 1116
Batch 13 de 1116
Batch 14 de 1116
Batch 15 de 1116
Batch 16 de 1116
Batch 17 de 1116
Batch 18 de 1116
Batch 19 de 1116
Batch 20 de 1116
Batch 21 de 1116
Batch 22 de 1116
Batch 23 de 1116
Batch 24 de 1116
Batch 25 de 1116
Batch 26 de 1116
Batch 27 de 1116
Batch 28 de 1116
Batch 29 de 1116
Batch 30 de 1116
Batch 31 de 1116
Batch 32 de 1116
Batch 33 de 1116
Batch 34 de 1116
Batch 35 de 1116
Batch 36 de 1116
Batch 37 de 1116
Batch 38 de 1116
Batch 39 de 1116
Batch 40 de 1116
Batch 41 de 1116
Batch 42 de 1116
Batch 43 de 1116
Batch 44 de 1116
Batch 45 de 1116
Batch 46 de 1116
Batch 47 de 1116
Batch 48 de 1116
Batch 49 de 1116
Batch 50 de 1116
Batch 51 de 1116
Batch 52 de 1116
Batch 53 de 1116
Batch 54 de 1116
Batch 55 de 1116
Batch 56 de 1116
Batch 57 de 1116
Batch 58 de 1116
Batch 5

##### Entrenamiento del modelo (80% de datos)

Utiliza:
- LR 1e-3
- weight_decay=0.01
- Dropout + Linear + Sigmoid

In [ ]:
for epoch in range(EPOCHS):
  trainBert(epoch)

Epoch: 0, iteración; 0 de 14277, Loss: 0.0793895661830902
Epoch: 0, iteración; 10 de 14277, Loss: 0.5832354068756104
Epoch: 0, iteración; 20 de 14277, Loss: 0.5882619380950928
Epoch: 0, iteración; 30 de 14277, Loss: 0.5497564315795899
Epoch: 0, iteración; 40 de 14277, Loss: 0.5226046085357666
Epoch: 0, iteración; 50 de 14277, Loss: 0.6374249458312988
Epoch: 0, iteración; 60 de 14277, Loss: 0.6370265483856201
Epoch: 0, iteración; 70 de 14277, Loss: 0.5931281089782715
Epoch: 0, iteración; 80 de 14277, Loss: 0.5791791915893555
Epoch: 0, iteración; 90 de 14277, Loss: 0.5664691925048828
Epoch: 0, iteración; 100 de 14277, Loss: 0.6570603847503662
Epoch: 0, iteración; 110 de 14277, Loss: 0.6109620571136475
Epoch: 0, iteración; 120 de 14277, Loss: 0.6524017333984375
Epoch: 0, iteración; 130 de 14277, Loss: 0.6393204689025879
Epoch: 0, iteración; 140 de 14277, Loss: 0.5577010154724121
Epoch: 0, iteración; 150 de 14277, Loss: 0.5565076351165772
Epoch: 0, iteración; 160 de 14277, Loss: 0.56684250

In [ ]:
try:
    torch.save(model.state_dict(), "./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_80_separados.pth")
    print("Modelo guardado correctamente")
except Exception as e:
    print(f"Error al guardar el modelo: {e}")

Modelo guardado correctamente


##### Evaluación del modelo

In [ ]:
def validationBert(epoch):
  model.eval()
  fin_targets=[]
  fin_outputs=[]
  num_iteraciones = len(testing_loader)
  with torch.no_grad():
    for i, data in enumerate(testing_loader, 0):
      ids = data['ids'].to(device)
      mask = data['mask'].to(device)
      token_type_ids = data['token_type_ids'].to(device)
      targets = data['target'].to(device)

      print(f"Batch {i} de {num_iteraciones}")

      outputs = model(ids, mask, token_type_ids)
      fin_targets.extend(targets.cpu().detach().numpy().tolist())
      fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
  return fin_outputs, fin_targets

**Nota:** Esta celda no se encuentra ejecutada porque solo se usa cuando la evaluación de un modelo se ha hecho en una sesión distinta a su entrenamiento. Si se hace en la misma sesión, no tenemos necesidad de cargar el modelo (que acabamos de entrenar). Por el contrario, si lo entrenamos "hoy" y lo evaluamos "mañana", en vez de reentrenarlo para poder evaluarlo, solo es necesario cargarlo.

In [ ]:
if os.path.exists("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_80_separados.pth"):
  model = BERTClass()
  if device == "cuda":
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_80_separados.pth"))
    print("Modelo cargado correctamente")
  else:
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_80_separados.pth", map_location="cpu"))
    print("Modelo cargado correctamente con CPU")

  model.eval()
  model.to(device)

Modelo cargado correctamente


In [ ]:
for epoch in range(EPOCHS):
  outputs, targets = validationBert(epoch)
  outputs = np.array(outputs) >= 0.5

  targets = np.array(targets).flatten().astype(int)

  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
  print(f"Accuracy Score = {accuracy}")
  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

Batch 0 de 447
Batch 1 de 447
Batch 2 de 447
Batch 3 de 447
Batch 4 de 447
Batch 5 de 447
Batch 6 de 447
Batch 7 de 447
Batch 8 de 447
Batch 9 de 447
Batch 10 de 447
Batch 11 de 447
Batch 12 de 447
Batch 13 de 447
Batch 14 de 447
Batch 15 de 447
Batch 16 de 447
Batch 17 de 447
Batch 18 de 447
Batch 19 de 447
Batch 20 de 447
Batch 21 de 447
Batch 22 de 447
Batch 23 de 447
Batch 24 de 447
Batch 25 de 447
Batch 26 de 447
Batch 27 de 447
Batch 28 de 447
Batch 29 de 447
Batch 30 de 447
Batch 31 de 447
Batch 32 de 447
Batch 33 de 447
Batch 34 de 447
Batch 35 de 447
Batch 36 de 447
Batch 37 de 447
Batch 38 de 447
Batch 39 de 447
Batch 40 de 447
Batch 41 de 447
Batch 42 de 447
Batch 43 de 447
Batch 44 de 447
Batch 45 de 447
Batch 46 de 447
Batch 47 de 447
Batch 48 de 447
Batch 49 de 447
Batch 50 de 447
Batch 51 de 447
Batch 52 de 447
Batch 53 de 447
Batch 54 de 447
Batch 55 de 447
Batch 56 de 447
Batch 57 de 447
Batch 58 de 447
Batch 59 de 447
Batch 60 de 447
Batch 61 de 447
Batch 62 de 447
Ba

###2.5.6 Modelo 2: RoBERTa

##### Generación Dataset y Dataloader

In [ ]:
training_loader, testing_loader = generate_loaders(datos, tokenizerRB,
                                                   restaurantsDataset,
                                                   DataCollatorRestaurant
                                                   )

FULL Dataset:(142768, 2)
TRAIN Dataset: (71384, 2)
TEST Dataset: (71384, 2)


##### Definición del modelo

In [ ]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.dropout = 0.2
        self.hidden_embd = 768
        self.output_layer = 1

        # Layers
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        #self.l2 = torch.nn.Linear(self.hidden_embd, 256)
        #self.l3 = torch.nn.Linear(256, 64)
        self.l4 = torch.nn.Dropout(self.dropout)
        # self.l5 = torch.nn.Linear(64, self.output_layer)
        self.l5 = torch.nn.Linear(self.hidden_embd, self.output_layer)
        self.l6 = torch.nn.Sigmoid()

    def forward(self, ids, mask, token_type_ids):
        _, output_1 = self.l1(ids, attention_mask=mask, token_type_ids=token_type_ids, return_dict=False)
        #output_2 = self.l2(output_1)
        #output_3 = self.l3(output_2)
        output_4 = self.l4(output_1)
        output = self.l5(output_4)
        output = self.l6(output)
        return output

model = RobertaClass()
model.to(device)

RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE,
                             weight_decay=0.01)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0.01
)

##### Función de entrenamiento

In [ ]:
def trainRoberta(epoch):
  model.train()
  loss_fn = torch.nn.BCELoss() #funcion perdida categorical cross entropy
  num_iteraciones = len(training_loader)
  sum_loss = 0

  for iteracion,data in enumerate(training_loader, 0):
    ids = data['ids'].to(device)
    mask = data['mask'].to(device)
    targets = data['target'].to(device)
    token_type_ids = data["token_type_ids"].to(device)

    output = model(ids, mask, token_type_ids)
    optimizer.zero_grad()

    perdida = loss_fn(output.squeeze(), targets)
    with torch.no_grad():
      sum_loss+=perdida
      if iteracion % PASOS_POR_INTERVALO == 0:
        print(f'Epoch: {epoch}, iteración; {iteracion:6} de {num_iteraciones}, Loss: {sum_loss.cpu().numpy()/PASOS_POR_INTERVALO}')
        sum_loss = 0

    optimizer.zero_grad()
    perdida.backward()
    optimizer.step()

##### Entrenamiento del modelo (30% de datos)

Utiliza:
- LR 1e-3
- weight_decay=0.01
- Dropout + Linear + Sigmoid

In [ ]:
for epoch in range(EPOCHS):
  trainRoberta(epoch)

Epoch: 0, iteración;      0 de 5354, Loss: 0.07095611095428467
Epoch: 0, iteración;     10 de 5354, Loss: 0.6335700035095215
Epoch: 0, iteración;     20 de 5354, Loss: 0.5745832443237304
Epoch: 0, iteración;     30 de 5354, Loss: 0.6549041748046875
Epoch: 0, iteración;     40 de 5354, Loss: 0.7494820117950439
Epoch: 0, iteración;     50 de 5354, Loss: 0.6497314453125
Epoch: 0, iteración;     60 de 5354, Loss: 0.5600525856018066
Epoch: 0, iteración;     70 de 5354, Loss: 0.5947575569152832
Epoch: 0, iteración;     80 de 5354, Loss: 0.627333688735962
Epoch: 0, iteración;     90 de 5354, Loss: 0.5397816658020019
Epoch: 0, iteración;    100 de 5354, Loss: 0.574896240234375
Epoch: 0, iteración;    110 de 5354, Loss: 0.568597412109375
Epoch: 0, iteración;    120 de 5354, Loss: 0.5479228496551514
Epoch: 0, iteración;    130 de 5354, Loss: 0.6033951759338378
Epoch: 0, iteración;    140 de 5354, Loss: 0.6093110561370849
Epoch: 0, iteración;    150 de 5354, Loss: 0.5871122360229493
Epoch: 0, ite

In [ ]:
try:
    torch.save(model.state_dict(), "./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_roberta_30_separados.pth")
    print("Modelo guardado correctamente")
except Exception as e:
    print(f"Error al guardar el modelo: {e}")

Modelo guardado correctamente


##### Evaluación del modelo

In [ ]:
def validationRoberta(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    num_iteraciones = len(testing_loader)
    with torch.no_grad():
        for i, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device)
            mask = data['mask'].to(device)
            token_type_ids = data['token_type_ids'].to(device)
            targets = data['target'].to(device)

            print(f"Iteración: {i:6} de {num_iteraciones}")

            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

**Nota:** Esta celda no se encuentra ejecutada porque solo se usa cuando la evaluación de un modelo se ha hecho en una sesión distinta a su entrenamiento. Si se hace en la misma sesión, no tenemos necesidad de cargar el modelo (que acabamos de entrenar). Por el contrario, si lo entrenamos "hoy" y lo evaluamos "mañana", en vez de reentrenarlo para poder evaluarlo, solo es necesario cargarlo.

In [ ]:
if os.path.exists("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_roberta_30_separados.pth"):
  model = RobertaClass()
  if device == "cuda":
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_roberta_30_separados.pth"))
    print("Modelo cargado correctamente")
  else:
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_roberta_30_separados.pth", map_location="cpu"))
    print("Modelo cargado correctamente con CPU")

  model.eval()
  model.to(device)

Modelo cargado correctamente


In [ ]:
for epoch in range(EPOCHS):
  outputs, targets = validationRoberta(epoch)
  outputs = np.array(outputs) >= 0.5
  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
  print(f"Accuracy Score = {accuracy}")
  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

Iteración:      0 de 1562
Iteración:      1 de 1562
Iteración:      2 de 1562
Iteración:      3 de 1562
Iteración:      4 de 1562
Iteración:      5 de 1562
Iteración:      6 de 1562
Iteración:      7 de 1562
Iteración:      8 de 1562
Iteración:      9 de 1562
Iteración:     10 de 1562
Iteración:     11 de 1562
Iteración:     12 de 1562
Iteración:     13 de 1562
Iteración:     14 de 1562
Iteración:     15 de 1562
Iteración:     16 de 1562
Iteración:     17 de 1562
Iteración:     18 de 1562
Iteración:     19 de 1562
Iteración:     20 de 1562
Iteración:     21 de 1562
Iteración:     22 de 1562
Iteración:     23 de 1562
Iteración:     24 de 1562
Iteración:     25 de 1562
Iteración:     26 de 1562
Iteración:     27 de 1562
Iteración:     28 de 1562
Iteración:     29 de 1562
Iteración:     30 de 1562
Iteración:     31 de 1562
Iteración:     32 de 1562
Iteración:     33 de 1562
Iteración:     34 de 1562
Iteración:     35 de 1562
Iteración:     36 de 1562
Iteración:     37 de 1562
Iteración:  

##### Entrenamiento del modelo (50% de datos)

Utiliza:
- LR 1e-3
- weight_decay=0.01
- Dropout + Linear + Sigmoid

In [ ]:
for epoch in range(EPOCHS):
  trainRoberta(epoch)

Epoch: 0, iteración;      0 de 8923, Loss: 0.06986327767372132
Epoch: 0, iteración;     10 de 8923, Loss: 0.9611250877380371
Epoch: 0, iteración;     20 de 8923, Loss: 0.6159377574920655
Epoch: 0, iteración;     30 de 8923, Loss: 0.6526411533355713
Epoch: 0, iteración;     40 de 8923, Loss: 0.7130374431610107
Epoch: 0, iteración;     50 de 8923, Loss: 0.6282275676727295
Epoch: 0, iteración;     60 de 8923, Loss: 0.5872012138366699
Epoch: 0, iteración;     70 de 8923, Loss: 0.6208059310913085
Epoch: 0, iteración;     80 de 8923, Loss: 0.5693597316741943
Epoch: 0, iteración;     90 de 8923, Loss: 0.522758960723877
Epoch: 0, iteración;    100 de 8923, Loss: 0.6788860321044922
Epoch: 0, iteración;    110 de 8923, Loss: 0.6283106803894043
Epoch: 0, iteración;    120 de 8923, Loss: 0.5955969333648682
Epoch: 0, iteración;    130 de 8923, Loss: 0.5371477127075195
Epoch: 0, iteración;    140 de 8923, Loss: 0.5649879932403564
Epoch: 0, iteración;    150 de 8923, Loss: 0.45634145736694337
Epoch: 

In [ ]:
try:
    torch.save(model.state_dict(), "./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_roberta_50_separados.pth")
    print("Modelo guardado correctamente")
except Exception as e:
    print(f"Error al guardar el modelo: {e}")

Modelo guardado correctamente


##### Evaluación del modelo

In [ ]:
def validationRoberta(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    num_iteraciones = len(testing_loader)
    with torch.no_grad():
        for i, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device)
            mask = data['mask'].to(device)
            token_type_ids = data['token_type_ids'].to(device)
            targets = data['target'].to(device)

            print(f"Iteración: {i:6} de {num_iteraciones}")

            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

**Nota:** Esta celda no se encuentra ejecutada porque solo se usa cuando la evaluación de un modelo se ha hecho en una sesión distinta a su entrenamiento. Si se hace en la misma sesión, no tenemos necesidad de cargar el modelo (que acabamos de entrenar). Por el contrario, si lo entrenamos "hoy" y lo evaluamos "mañana", en vez de reentrenarlo para poder evaluarlo, solo es necesario cargarlo.

In [ ]:
if os.path.exists("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_roberta_50_separados.pth"):
  model = RobertaClass()
  if device == "cuda":
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_roberta_50_separados.pth"))
    print("Modelo cargado correctamente")
  else:
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_roberta_50_separados.pth", map_location="cpu"))
    print("Modelo cargado correctamente con CPU")

  model.eval()
  model.to(device)

Modelo cargado correctamente


In [ ]:
for epoch in range(EPOCHS):
  outputs, targets = validationRoberta(epoch)
  outputs = np.array(outputs) >= 0.5
  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
  print(f"Accuracy Score = {accuracy}")
  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

Iteración:      0 de 1116
Iteración:      1 de 1116
Iteración:      2 de 1116
Iteración:      3 de 1116
Iteración:      4 de 1116
Iteración:      5 de 1116
Iteración:      6 de 1116
Iteración:      7 de 1116
Iteración:      8 de 1116
Iteración:      9 de 1116
Iteración:     10 de 1116
Iteración:     11 de 1116
Iteración:     12 de 1116
Iteración:     13 de 1116
Iteración:     14 de 1116
Iteración:     15 de 1116
Iteración:     16 de 1116
Iteración:     17 de 1116
Iteración:     18 de 1116
Iteración:     19 de 1116
Iteración:     20 de 1116
Iteración:     21 de 1116
Iteración:     22 de 1116
Iteración:     23 de 1116
Iteración:     24 de 1116
Iteración:     25 de 1116
Iteración:     26 de 1116
Iteración:     27 de 1116
Iteración:     28 de 1116
Iteración:     29 de 1116
Iteración:     30 de 1116
Iteración:     31 de 1116
Iteración:     32 de 1116
Iteración:     33 de 1116
Iteración:     34 de 1116
Iteración:     35 de 1116
Iteración:     36 de 1116
Iteración:     37 de 1116
Iteración:  

##2.6 Aproximación 3: Utilizando Siamese

En esta aproximación se plantean arquitecturas siamesas. Se obtendrán de manera independiente los *embeddings* de cada uno de los textos y estos *embeddings* serán lo que se usen para obtener las salidas bien sea mediante distancia coseno o alimentando directamente la red con la concatenación de los *embeddings*. Para este caso, se hará uso tanto de `Review1` como de `Review2` y `Sentimiento`.

###2.6.1 Carga de datos

En este punto, se cargan los datos preprocesados para esta tarea, es decir, los obtenidos del apartado 1.3.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
datos = pd.read_feather("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/valoraciones_Siamese_train.feather")
datos

,Review1,Review2,Sentimiento
0,Nice brunch,Good place,1
1,Worlds best hot cocolate!,Great place!,1
2,Great food and service,"Rated on 7,0!",1
3,Delicious and friendly,Great little French Bistro!,0
4,Late dinner,Very nice dinner,0
...,...,...,...
65314,"Very nice food, great location",Great calamari,1
65315,Great Bavarian food in a cozy place,Traditional restaurant,1
65316,Italy traditions in Vienna,Very nice restaurant,1
65317,3 visits over 4 days,Great food & atmosphere,1


###2.6.2 Definición tokenizadores y modelos

In [ ]:
tokenizerB = BertTokenizer.from_pretrained("bert-base-uncased")
modelB = BertModel.from_pretrained("bert-base-uncased")
configB = AutoConfig.from_pretrained("bert-base-uncased")

###2.6.3 Función de generación de conjuntos de train y test: DataLoaders

In [ ]:
def generate_loaders(dataframe, tokenizer, dataset, collator):
  train_dataset = dataframe.sample(frac=TRAIN_SIZE, random_state=0)
  test_dataset = dataframe.drop(train_dataset.index).reset_index(drop=True)
  train_dataset = train_dataset.reset_index(drop=True)

  print(f"FULL Dataset:{dataframe.shape}")
  print(f"TRAIN Dataset: {train_dataset.shape}")
  print(f"TEST Dataset: {test_dataset.shape}")

  training_set = dataset(train_dataset, MAX_LEN)
  testing_set = dataset(test_dataset, MAX_LEN)
  dc = collator(tokenizer, MAX_LEN)


  train_params = {'batch_size': TRAIN_BATCH_SIZE,
                  'shuffle': True,
                  'num_workers': 0,
                  'collate_fn': dc
                  }

  test_params = { 'batch_size': VALID_BATCH_SIZE,
                  'shuffle': True,
                  'num_workers': 0,
                  'collate_fn': dc
                }

  training_loader = DataLoader(training_set, **train_params)
  testing_loader = DataLoader(testing_set, **test_params)
  return training_loader, testing_loader

###2.6.4 Definición restaurantsSiameseDataset y DataCollatorSiamese

In [ ]:
class restaurantsSiameseDataset(Dataset):
  def __init__(self, dataframe, max_len):
    self.data = dataframe
    self.texto1 = dataframe.Review1
    self.texto2 = dataframe.Review2
    self.targets = self.data.Sentimiento
    self.max_len = max_len

  def __len__(self):
    return len(self.texto1)

  def __getitem__(self, index):
    comment_text_1 = self.texto1[index]
    comment_text_2 = self.texto2[index]
    target = self.targets[index]

    return {
        "texto1": comment_text_1,
        "texto2": comment_text_2,
        "target": target
    }

In [ ]:
datasetSiamese = restaurantsSiameseDataset(datos, MAX_LEN)
datasetSiamese.__getitem__(0)

{'texto1': 'Nice brunch', 'texto2': 'Good place', 'target': 1}

In [ ]:
class DataCollatorSiamese:
    def __init__(self, tokenizer, max_len=512):
      self.tokenizer = tokenizer
      self.max_len = max_len

    def __call__(self, input_batch):
      data_frame = pd.DataFrame(input_batch)
      batch_dict = {column: data_frame[column].tolist() for column in data_frame}

      encoder_input1 = self.tokenizer(
          batch_dict["texto1"],
          None,
          add_special_tokens=True,
          truncation=True,
          max_length=self.max_len,
          pad_to_max_length=True,
          return_tensors = "pt",
          return_token_type_ids=True
          )
      encoder_input2 = self.tokenizer(
          batch_dict["texto2"],
          None,
          add_special_tokens=True,
          truncation=True,
          max_length=self.max_len,
          pad_to_max_length=True,
          return_tensors = "pt",
          return_token_type_ids=True
          )

      return {
      'ids1': encoder_input1["input_ids"],
      'mask1': encoder_input1["attention_mask"],
      "token_type_ids1": encoder_input1["token_type_ids"],
      'ids2': encoder_input2["input_ids"],
      'mask2': encoder_input2["attention_mask"],
      "token_type_ids2": encoder_input2["token_type_ids"],
      "target": torch.Tensor(batch_dict["target"]),
    }

In [ ]:
training_loader, testing_loader = generate_loaders(datos, tokenizerB,
                                                   restaurantsSiameseDataset,
                                                   DataCollatorSiamese
                                                   )

FULL Dataset:(65319, 3)
TRAIN Dataset: (32660, 3)
TEST Dataset: (32659, 3)


###2.6.5 CosineSimilarity Approach

En esta aproximación, se emplea la distancia coseno como característica de similaridad para que el modelo aprenda. Asimismo, mencionar que se utiliza BERT como modelo preentrenado.

###### Definición del modelo

In [ ]:
class BERTClassSiamese(torch.nn.Module):
  def __init__(self):
    super(BERTClassSiamese, self).__init__()
    self.dropout = 0.2
    self.hidden_embd = 768
    self.output_layer = 1

    # Layers
    self.l1 = BertModel.from_pretrained('bert-base-uncased')
    self.l2 = torch.nn.Linear(self.hidden_embd, self.hidden_embd)
    self.l3 = torch.nn.Dropout(self.dropout)
    self.l4 = torch.nn.CosineSimilarity(dim=1)
    self.l5 = torch.nn.Linear(1, self.output_layer)

  def forward(self, ids_1, mask_1, token_type_ids_1, ids_2, mask_2, token_type_ids_2):
    _, last_hidden_state_b = self.l1(ids_2, attention_mask=mask_2, token_type_ids=token_type_ids_2, return_dict=False)
    _, last_hidden_state_a = self.l1(ids_1, attention_mask=mask_1, token_type_ids=token_type_ids_1, return_dict=False)

    x_a, x_b = self.l2(last_hidden_state_a), self.l2(last_hidden_state_b)
    x_a, x_b = F.gelu(self.l3(x_a)), F.gelu(self.l3(x_b))
    sem_sim = self.l4(x_a, x_b).unsqueeze(-1) # Mejor hacer un concat
    weighted_sem_sim = self.l5(sem_sim)
    return weighted_sem_sim

model = BERTClassSiamese()
model.to(device)

BERTClassSiamese(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE,
                             weight_decay=0.01)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0.01
)

###### Entrenamiento del modelo

Utiliza:
- LR 1e-3
- weight_decay=0.01
- Linear + Dropout + CosineSimilarity + Linear

In [ ]:
def trainBERTSiamese(epoch):
  model.train()
  loss_fn = torch.nn.BCEWithLogitsLoss()
  num_iteraciones = len(training_loader)
  sum_loss = 0

  for iteracion,data in enumerate(training_loader, 0):
    ids1 = data['ids1'].to(device)
    ids2 = data['ids2'].to(device)
    mask1 = data['mask1'].to(device)
    mask2 = data['mask2'].to(device)
    token_type_ids1 = data["token_type_ids1"].to(device)
    token_type_ids2 = data["token_type_ids2"].to(device)
    targets = data['target'].to(device)

    output = model(ids1, mask1, token_type_ids1, ids2, mask2, token_type_ids2)
    optimizer.zero_grad()

    perdida = loss_fn(output.squeeze(), targets)
    with torch.no_grad():
      sum_loss+=perdida
      if iteracion % PASOS_POR_INTERVALO == 0:
        print(f'Epoch: {epoch}, iteración; {iteracion:6} de {num_iteraciones}, Loss: {sum_loss.cpu().numpy()/PASOS_POR_INTERVALO}')
        sum_loss = 0

    optimizer.zero_grad()
    perdida.backward()
    optimizer.step()

In [ ]:
for epoch in range(EPOCHS):
  trainBERTSiamese(epoch)

Epoch: 0, iteración;      0 de 2450, Loss: 0.08972876071929932
Epoch: 0, iteración;     10 de 2450, Loss: 0.8161079406738281
Epoch: 0, iteración;     20 de 2450, Loss: 0.8100442886352539
Epoch: 0, iteración;     30 de 2450, Loss: 0.8250888824462891
Epoch: 0, iteración;     40 de 2450, Loss: 0.7858126640319825
Epoch: 0, iteración;     50 de 2450, Loss: 0.7962770938873291
Epoch: 0, iteración;     60 de 2450, Loss: 0.8124420166015625
Epoch: 0, iteración;     70 de 2450, Loss: 0.7808439254760742
Epoch: 0, iteración;     80 de 2450, Loss: 0.771768045425415
Epoch: 0, iteración;     90 de 2450, Loss: 0.7912948131561279
Epoch: 0, iteración;    100 de 2450, Loss: 0.7919481277465821
Epoch: 0, iteración;    110 de 2450, Loss: 0.7901940822601319
Epoch: 0, iteración;    120 de 2450, Loss: 0.7357278823852539
Epoch: 0, iteración;    130 de 2450, Loss: 0.7539856910705567
Epoch: 0, iteración;    140 de 2450, Loss: 0.7280426502227784
Epoch: 0, iteración;    150 de 2450, Loss: 0.7667979240417481
Epoch: 0

In [ ]:
try:
    torch.save(model.state_dict(), "./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_siamese_bert_cosine_30.pth")
    print("Modelo guardado correctamente")
except Exception as e:
    print(f"Error al guardar el modelo: {e}")

Modelo guardado correctamente


###### Evaluación

In [ ]:
def validationSiamese(epoch):
  model.eval()
  fin_targets=[]
  fin_outputs=[]
  num_iteraciones = len(testing_loader)
  with torch.no_grad():
    for i, data in enumerate(testing_loader, 0):
      ids1 = data['ids1'].to(device)
      ids2 = data['ids2'].to(device)
      mask1 = data['mask1'].to(device)
      mask2 = data['mask2'].to(device)
      token_type_ids1 = data["token_type_ids1"].to(device)
      token_type_ids2 = data["token_type_ids2"].to(device)
      targets = data['target'].to(device)

      print(f"Iteración: {i:6} de {num_iteraciones}")

      outputs = model(ids1, mask1, token_type_ids1, ids2, mask2, token_type_ids2)
      fin_targets.extend(targets.cpu().detach().numpy().tolist())
      fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
  return fin_outputs, fin_targets

**Nota:** Esta celda no se encuentra ejecutada porque solo se usa cuando la evaluación de un modelo se ha hecho en una sesión distinta a su entrenamiento. Si se hace en la misma sesión, no tenemos necesidad de cargar el modelo (que acabamos de entrenar). Por el contrario, si lo entrenamos "hoy" y lo evaluamos "mañana", en vez de reentrenarlo para poder evaluarlo, solo es necesario cargarlo.

In [ ]:
if os.path.exists("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_siamese_bert_cosine_30.pth"):
  model = BERTClassSiamese()
  if device == "cuda":
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_siamese_bert_cosine_30.pth"))
    print("Modelo cargado correctamente")
  else:
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_siamese_bert_cosine_30.pth", map_location="cpu"))
    print("Modelo cargado correctamente con CPU")

  model.eval()
  model.to(device)

Modelo cargado correctamente


In [ ]:
for epoch in range(EPOCHS):
  outputs, targets = validationSiamese(epoch)
  outputs = np.array(outputs) >= 0.5
  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
  print(f"Accuracy Score = {accuracy}")
  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

Iteración:      0 de 715
Iteración:      1 de 715
Iteración:      2 de 715
Iteración:      3 de 715
Iteración:      4 de 715
Iteración:      5 de 715
Iteración:      6 de 715
Iteración:      7 de 715
Iteración:      8 de 715
Iteración:      9 de 715
Iteración:     10 de 715
Iteración:     11 de 715
Iteración:     12 de 715
Iteración:     13 de 715
Iteración:     14 de 715
Iteración:     15 de 715
Iteración:     16 de 715
Iteración:     17 de 715
Iteración:     18 de 715
Iteración:     19 de 715
Iteración:     20 de 715
Iteración:     21 de 715
Iteración:     22 de 715
Iteración:     23 de 715
Iteración:     24 de 715
Iteración:     25 de 715
Iteración:     26 de 715
Iteración:     27 de 715
Iteración:     28 de 715
Iteración:     29 de 715
Iteración:     30 de 715
Iteración:     31 de 715
Iteración:     32 de 715
Iteración:     33 de 715
Iteración:     34 de 715
Iteración:     35 de 715
Iteración:     36 de 715
Iteración:     37 de 715
Iteración:     38 de 715
Iteración:     39 de 715


###2.6.6 ConcatEmbeddings Approach

En esta aproximación no se emplea la distancia coseno ni ninguna otra medida, si no que, en su lugar, se alimenta directamente la cabeza clasificadora con la concatenación de los dos *embeddings*.

###### Definición del modelo

In [ ]:
class BERTSiameseConcat(torch.nn.Module):
  def __init__(self):
    super(BERTSiameseConcat, self).__init__()
    self.dropout = 0.2
    self.hidden_embd = 768
    self.output_layer = 1

    # Layers
    self.l1 = BertModel.from_pretrained('bert-base-uncased')

    self.l2 = torch.nn.Linear(self.hidden_embd  * 2, self.hidden_embd)
    self.a2 = torch.nn.LeakyReLU()
    self.l3 = torch.nn.Dropout(self.dropout)

    self.l4 = torch.nn.Linear(self.hidden_embd, 512)
    self.a4 = torch.nn.LeakyReLU()

    self.l5 = torch.nn.Linear(512, self.output_layer)
    self.a5 = torch.nn.Sigmoid()

  def forward(self, ids_1, mask_1, token_type_ids_1, ids_2, mask_2, token_type_ids_2):
    _, last_hidden_state_b = self.l1(ids_2, attention_mask=mask_2, token_type_ids=token_type_ids_2, return_dict=False)
    _, last_hidden_state_a = self.l1(ids_1, attention_mask=mask_1, token_type_ids=token_type_ids_1, return_dict=False)
    last_hidden_state_final = torch.cat([last_hidden_state_a, last_hidden_state_b], dim=1)

    outputl2 = self.l2(last_hidden_state_final)
    outputa2 = self.a2(outputl2)

    #outputl3 = self.l3(outputl2)
    outputl3 = self.l3(outputa2)

    outputl4 = self.l4(outputl3)
    outputa4 = self.a4(outputl4)

    #outputl5 = self.l5(outputl4)
    outputl5 = self.l5(outputa4)
    outputa5 = self.a5(outputl5)

    #return outputl5
    return outputa5

model = BERTSiameseConcat()
model.to(device)

BERTSiameseConcat(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE,
                             weight_decay=0.001)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 1e-07
    maximize: False
    weight_decay: 0.001
)

###### Función de entrenamiento

In [ ]:
def trainBERTSiameseConcat(epoch):
  model.train()
  #loss_fn = torch.nn.BCEWithLogitsLoss()
  #loss_fn = torch.nn.CrossEntropyLoss()
  #loss_fn = torch.nn.MSELoss()
  loss_fn = torch.nn.BCELoss()
  num_iteraciones = len(training_loader)
  sum_loss = 0

  for iteracion,data in enumerate(training_loader, 0):
    ids1 = data['ids1'].to(device)
    ids2 = data['ids2'].to(device)
    mask1 = data['mask1'].to(device)
    mask2 = data['mask2'].to(device)
    token_type_ids1 = data["token_type_ids1"].to(device)
    token_type_ids2 = data["token_type_ids2"].to(device)
    targets = data['target'].to(device)

    output = model(ids1, mask1, token_type_ids1, ids2, mask2, token_type_ids2)
    optimizer.zero_grad()

    perdida = loss_fn(output.squeeze(), targets)
    with torch.no_grad():
      sum_loss+=perdida
      if iteracion % PASOS_POR_INTERVALO == 0:
        print(f'Epoch: {epoch}, iteración: {iteracion} de {num_iteraciones}, Loss: {sum_loss.cpu().numpy()/PASOS_POR_INTERVALO}')
        sum_loss = 0

    optimizer.zero_grad()
    perdida.backward()
    optimizer.step()

###### Entrenamiento del modelo (30% de datos)

Utiliza:
- LR 1e-3
- weight_decay=0.001
- Linear + Dropout + Linear + Linear + Sigmoid

In [ ]:
for epoch in range(EPOCHS):
  trainBERTSiameseConcat(epoch)

Epoch: 0, iteración: 0 de 2450, Loss: 0.0777856945991516
Epoch: 0, iteración: 10 de 2450, Loss: 2.2393213272094727
Epoch: 0, iteración: 20 de 2450, Loss: 0.8985054969787598
Epoch: 0, iteración: 30 de 2450, Loss: 0.7053309440612793
Epoch: 0, iteración: 40 de 2450, Loss: 0.6196593284606934
Epoch: 0, iteración: 50 de 2450, Loss: 0.6945080280303955
Epoch: 0, iteración: 60 de 2450, Loss: 0.5722953796386718
Epoch: 0, iteración: 70 de 2450, Loss: 0.6320746421813965
Epoch: 0, iteración: 80 de 2450, Loss: 0.6995283126831054
Epoch: 0, iteración: 90 de 2450, Loss: 0.5264666080474854
Epoch: 0, iteración: 100 de 2450, Loss: 0.5602176666259766
Epoch: 0, iteración: 110 de 2450, Loss: 0.5656137943267823
Epoch: 0, iteración: 120 de 2450, Loss: 0.5682831764221191
Epoch: 0, iteración: 130 de 2450, Loss: 0.6760453701019287
Epoch: 0, iteración: 140 de 2450, Loss: 0.634089994430542
Epoch: 0, iteración: 150 de 2450, Loss: 0.5047501564025879
Epoch: 0, iteración: 160 de 2450, Loss: 0.6530325412750244
Epoch: 0,

In [ ]:
try:
  torch.save(model.state_dict(), "./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_siamese_bert_concat_30.pth")
  print("Modelo guardado correctamente")
except Exception as e:
    print(f"Error al guardar el modelo: {e}")

Modelo guardado correctamente


###### Evaluación del modelo

In [ ]:
def validationBERTSiameseConcat(epoch):
  model.eval()
  fin_targets=[]
  fin_outputs=[]
  num_iteraciones = len(testing_loader)
  with torch.no_grad():
    for i, data in enumerate(testing_loader, 0):
      ids1 = data['ids1'].to(device)
      ids2 = data['ids2'].to(device)
      mask1 = data['mask1'].to(device)
      mask2 = data['mask2'].to(device)
      token_type_ids1 = data["token_type_ids1"].to(device)
      token_type_ids2 = data["token_type_ids2"].to(device)
      targets = data['target'].to(device)

      print(f"Iteración: {i} de {num_iteraciones}")

      outputs = model(ids1, mask1, token_type_ids1, ids2, mask2, token_type_ids2)
      fin_targets.extend(targets.cpu().detach().numpy().tolist())
      fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
  return fin_outputs, fin_targets

**Nota:** Esta celda no se encuentra ejecutada porque solo se usa cuando la evaluación de un modelo se ha hecho en una sesión distinta a su entrenamiento. Si se hace en la misma sesión, no tenemos necesidad de cargar el modelo (que acabamos de entrenar). Por el contrario, si lo entrenamos "hoy" y lo evaluamos "mañana", en vez de reentrenarlo para poder evaluarlo, solo es necesario cargarlo.

In [ ]:
if os.path.exists("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_siamese_bert_concat_30.pth"):
  model = BERTSiameseConcat()
  if device == "cuda":
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_siamese_bert_concat_30.pth"))
    print("Modelo cargado correctamente")
  else:
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_siamese_bert_concat_30.pth", map_location="cpu"))
    print("Modelo cargado correctamente con CPU")

  model.eval()
  model.to(device)

Modelo cargado correctamente


In [ ]:
for epoch in range(EPOCHS):
  outputs, targets = validationBERTSiameseConcat(epoch)
  outputs = np.array(outputs) >= 0.5
  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
  print(f"Accuracy Score = {accuracy}")
  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

Iteración: 0 de 715
Iteración: 1 de 715
Iteración: 2 de 715
Iteración: 3 de 715
Iteración: 4 de 715
Iteración: 5 de 715
Iteración: 6 de 715
Iteración: 7 de 715
Iteración: 8 de 715
Iteración: 9 de 715
Iteración: 10 de 715
Iteración: 11 de 715
Iteración: 12 de 715
Iteración: 13 de 715
Iteración: 14 de 715
Iteración: 15 de 715
Iteración: 16 de 715
Iteración: 17 de 715
Iteración: 18 de 715
Iteración: 19 de 715
Iteración: 20 de 715
Iteración: 21 de 715
Iteración: 22 de 715
Iteración: 23 de 715
Iteración: 24 de 715
Iteración: 25 de 715
Iteración: 26 de 715
Iteración: 27 de 715
Iteración: 28 de 715
Iteración: 29 de 715
Iteración: 30 de 715
Iteración: 31 de 715
Iteración: 32 de 715
Iteración: 33 de 715
Iteración: 34 de 715
Iteración: 35 de 715
Iteración: 36 de 715
Iteración: 37 de 715
Iteración: 38 de 715
Iteración: 39 de 715
Iteración: 40 de 715
Iteración: 41 de 715
Iteración: 42 de 715
Iteración: 43 de 715
Iteración: 44 de 715
Iteración: 45 de 715
Iteración: 46 de 715
Iteración: 47 de 715
It

###### Entrenamiento del modelo (50% de datos) LR = 1e-7

In [ ]:
for epoch in range(EPOCHS):
  trainBERTSiameseConcat(epoch)

Epoch: 0, iteración: 0 de 4083, Loss: 0.06998984813690186
Epoch: 0, iteración: 10 de 4083, Loss: 0.7256251811981201
Epoch: 0, iteración: 20 de 4083, Loss: 0.7244583606719971
Epoch: 0, iteración: 30 de 4083, Loss: 0.7038580894470214
Epoch: 0, iteración: 40 de 4083, Loss: 0.7087534427642822
Epoch: 0, iteración: 50 de 4083, Loss: 0.7203779220581055
Epoch: 0, iteración: 60 de 4083, Loss: 0.7072153091430664
Epoch: 0, iteración: 70 de 4083, Loss: 0.7014464855194091
Epoch: 0, iteración: 80 de 4083, Loss: 0.7164917945861816
Epoch: 0, iteración: 90 de 4083, Loss: 0.7073198318481445
Epoch: 0, iteración: 100 de 4083, Loss: 0.7067512989044189
Epoch: 0, iteración: 110 de 4083, Loss: 0.7066843986511231
Epoch: 0, iteración: 120 de 4083, Loss: 0.6981516361236573
Epoch: 0, iteración: 130 de 4083, Loss: 0.7079751014709472
Epoch: 0, iteración: 140 de 4083, Loss: 0.6923242568969726
Epoch: 0, iteración: 150 de 4083, Loss: 0.6968338966369629
Epoch: 0, iteración: 160 de 4083, Loss: 0.6899566650390625
Epoch: 

In [ ]:
try:
  torch.save(model.state_dict(), "./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_siamese_bert_concat_lr7_50.pth")
  print("Modelo guardado correctamente")
except Exception as e:
    print(f"Error al guardar el modelo: {e}")

Modelo guardado correctamente


###### Evaluación del modelo

In [ ]:
def validationBERTSiameseConcat(epoch):
  model.eval()
  fin_targets=[]
  fin_outputs=[]
  num_iteraciones = len(testing_loader)
  with torch.no_grad():
    for i, data in enumerate(testing_loader, 0):
      ids1 = data['ids1'].to(device)
      ids2 = data['ids2'].to(device)
      mask1 = data['mask1'].to(device)
      mask2 = data['mask2'].to(device)
      token_type_ids1 = data["token_type_ids1"].to(device)
      token_type_ids2 = data["token_type_ids2"].to(device)
      targets = data['target'].to(device)

      print(f"Iteración: {i} de {num_iteraciones}")

      outputs = model(ids1, mask1, token_type_ids1, ids2, mask2, token_type_ids2)
      fin_targets.extend(targets.cpu().detach().numpy().tolist())
      fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
  return fin_outputs, fin_targets

**Nota:** Esta celda no se encuentra ejecutada porque solo se usa cuando la evaluación de un modelo se ha hecho en una sesión distinta a su entrenamiento. Si se hace en la misma sesión, no tenemos necesidad de cargar el modelo (que acabamos de entrenar). Por el contrario, si lo entrenamos "hoy" y lo evaluamos "mañana", en vez de reentrenarlo para poder evaluarlo, solo es necesario cargarlo.

In [ ]:
if os.path.exists("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_siamese_bert_concat_lr7_50.pth"):
  model = BERTSiameseConcat()
  if device == "cuda":
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_siamese_bert_concat_lr7_50.pth"))
    print("Modelo cargado correctamente")
  else:
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_siamese_bert_concat_lr7_50.pth", map_location="cpu"))
    print("Modelo cargado correctamente con CPU")

  model.eval()
  model.to(device)

Modelo cargado correctamente


In [ ]:
for epoch in range(EPOCHS):
  outputs, targets = validationBERTSiameseConcat(epoch)
  outputs = np.array(outputs) >= 0.5
  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
  print(f"Accuracy Score = {accuracy}")
  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

Iteración: 0 de 511
Iteración: 1 de 511
Iteración: 2 de 511
Iteración: 3 de 511
Iteración: 4 de 511
Iteración: 5 de 511
Iteración: 6 de 511
Iteración: 7 de 511
Iteración: 8 de 511
Iteración: 9 de 511
Iteración: 10 de 511
Iteración: 11 de 511
Iteración: 12 de 511
Iteración: 13 de 511
Iteración: 14 de 511
Iteración: 15 de 511
Iteración: 16 de 511
Iteración: 17 de 511
Iteración: 18 de 511
Iteración: 19 de 511
Iteración: 20 de 511
Iteración: 21 de 511
Iteración: 22 de 511
Iteración: 23 de 511
Iteración: 24 de 511
Iteración: 25 de 511
Iteración: 26 de 511
Iteración: 27 de 511
Iteración: 28 de 511
Iteración: 29 de 511
Iteración: 30 de 511
Iteración: 31 de 511
Iteración: 32 de 511
Iteración: 33 de 511
Iteración: 34 de 511
Iteración: 35 de 511
Iteración: 36 de 511
Iteración: 37 de 511
Iteración: 38 de 511
Iteración: 39 de 511
Iteración: 40 de 511
Iteración: 41 de 511
Iteración: 42 de 511
Iteración: 43 de 511
Iteración: 44 de 511
Iteración: 45 de 511
Iteración: 46 de 511
Iteración: 47 de 511
It

###### Entrenamiento del modelo (50% de datos) Loss MSE y lr = 1e-7

In [ ]:
for epoch in range(EPOCHS):
  trainBERTSiameseConcat(epoch)

Epoch: 0, iteración: 0 de 4083, Loss: 0.09142661690711976
Epoch: 0, iteración: 10 de 4083, Loss: 0.9750309944152832
Epoch: 0, iteración: 20 de 4083, Loss: 0.8977389335632324
Epoch: 0, iteración: 30 de 4083, Loss: 1.0117053031921386
Epoch: 0, iteración: 40 de 4083, Loss: 0.910208797454834
Epoch: 0, iteración: 50 de 4083, Loss: 0.9546347618103027
Epoch: 0, iteración: 60 de 4083, Loss: 0.8645976066589356
Epoch: 0, iteración: 70 de 4083, Loss: 0.8153607368469238
Epoch: 0, iteración: 80 de 4083, Loss: 0.8270837783813476
Epoch: 0, iteración: 90 de 4083, Loss: 0.8262016296386718
Epoch: 0, iteración: 100 de 4083, Loss: 0.7459123611450196
Epoch: 0, iteración: 110 de 4083, Loss: 0.7221677780151368
Epoch: 0, iteración: 120 de 4083, Loss: 0.6665257930755615
Epoch: 0, iteración: 130 de 4083, Loss: 0.7931793212890625
Epoch: 0, iteración: 140 de 4083, Loss: 0.7266057014465332
Epoch: 0, iteración: 150 de 4083, Loss: 0.8019199371337891
Epoch: 0, iteración: 160 de 4083, Loss: 0.6706379890441895
Epoch: 0

In [ ]:
try:
  torch.save(model.state_dict(), "./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_siamese_bert_concat_lr7_MSE_50.pth")
  print("Modelo guardado correctamente")
except Exception as e:
    print(f"Error al guardar el modelo: {e}")

Modelo guardado correctamente


###### Evaluación del modelo

In [ ]:
def validationBERTSiameseConcat(epoch):
  model.eval()
  fin_targets=[]
  fin_outputs=[]
  num_iteraciones = len(testing_loader)
  with torch.no_grad():
    for i, data in enumerate(testing_loader, 0):
      ids1 = data['ids1'].to(device)
      ids2 = data['ids2'].to(device)
      mask1 = data['mask1'].to(device)
      mask2 = data['mask2'].to(device)
      token_type_ids1 = data["token_type_ids1"].to(device)
      token_type_ids2 = data["token_type_ids2"].to(device)
      targets = data['target'].to(device)

      print(f"Iteración: {i} de {num_iteraciones}")

      outputs = model(ids1, mask1, token_type_ids1, ids2, mask2, token_type_ids2)
      fin_targets.extend(targets.cpu().detach().numpy().tolist())
      fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
  return fin_outputs, fin_targets

**Nota:** Esta celda no se encuentra ejecutada porque solo se usa cuando la evaluación de un modelo se ha hecho en una sesión distinta a su entrenamiento. Si se hace en la misma sesión, no tenemos necesidad de cargar el modelo (que acabamos de entrenar). Por el contrario, si lo entrenamos "hoy" y lo evaluamos "mañana", en vez de reentrenarlo para poder evaluarlo, solo es necesario cargarlo.

In [ ]:
if os.path.exists("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_siamese_bert_concat_lr7_MSE_50.pth"):
  model = BERTSiameseConcat()
  if device == "cuda":
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_siamese_bert_concat_lr7_MSE_50.pth"))
    print("Modelo cargado correctamente")
  else:
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_siamese_bert_concat_lr7_MSE_50.pth", map_location="cpu"))
    print("Modelo cargado correctamente con CPU")

  model.eval()
  model.to(device)

Modelo cargado correctamente


In [ ]:
for epoch in range(EPOCHS):
  outputs, targets = validationBERTSiameseConcat(epoch)
  outputs = np.array(outputs) >= 0.5
  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
  print(f"Accuracy Score = {accuracy}")
  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

Iteración: 0 de 511
Iteración: 1 de 511
Iteración: 2 de 511
Iteración: 3 de 511
Iteración: 4 de 511
Iteración: 5 de 511
Iteración: 6 de 511
Iteración: 7 de 511
Iteración: 8 de 511
Iteración: 9 de 511
Iteración: 10 de 511
Iteración: 11 de 511
Iteración: 12 de 511
Iteración: 13 de 511
Iteración: 14 de 511
Iteración: 15 de 511
Iteración: 16 de 511
Iteración: 17 de 511
Iteración: 18 de 511
Iteración: 19 de 511
Iteración: 20 de 511
Iteración: 21 de 511
Iteración: 22 de 511
Iteración: 23 de 511
Iteración: 24 de 511
Iteración: 25 de 511
Iteración: 26 de 511
Iteración: 27 de 511
Iteración: 28 de 511
Iteración: 29 de 511
Iteración: 30 de 511
Iteración: 31 de 511
Iteración: 32 de 511
Iteración: 33 de 511
Iteración: 34 de 511
Iteración: 35 de 511
Iteración: 36 de 511
Iteración: 37 de 511
Iteración: 38 de 511
Iteración: 39 de 511
Iteración: 40 de 511
Iteración: 41 de 511
Iteración: 42 de 511
Iteración: 43 de 511
Iteración: 44 de 511
Iteración: 45 de 511
Iteración: 46 de 511
Iteración: 47 de 511
It

###### Loss BCE + LeakyRelu + Sigmoid

In [ ]:
for epoch in range(EPOCHS):
  trainBERTSiameseConcat(epoch)

Epoch: 0, iteración: 0 de 4083, Loss: 0.06848677396774291
Epoch: 0, iteración: 10 de 4083, Loss: 0.6721079349517822
Epoch: 0, iteración: 20 de 4083, Loss: 0.6728714942932129
Epoch: 0, iteración: 30 de 4083, Loss: 0.6785274982452393
Epoch: 0, iteración: 40 de 4083, Loss: 0.6701336860656738
Epoch: 0, iteración: 50 de 4083, Loss: 0.6746286869049072
Epoch: 0, iteración: 60 de 4083, Loss: 0.6716646671295166
Epoch: 0, iteración: 70 de 4083, Loss: 0.6796284675598144
Epoch: 0, iteración: 80 de 4083, Loss: 0.6731937408447266
Epoch: 0, iteración: 90 de 4083, Loss: 0.675541877746582


KeyboardInterrupt: 

In [ ]:
try:
  torch.save(model.state_dict(), "./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_siamese_bert_concat_lr7_BCE_50.pth")
  print("Modelo guardado correctamente")
except Exception as e:
    print(f"Error al guardar el modelo: {e}")

###### Evaluación del modelo

In [ ]:
def validationBERTSiameseConcat(epoch):
  model.eval()
  fin_targets=[]
  fin_outputs=[]
  num_iteraciones = len(testing_loader)
  with torch.no_grad():
    for i, data in enumerate(testing_loader, 0):
      ids1 = data['ids1'].to(device)
      ids2 = data['ids2'].to(device)
      mask1 = data['mask1'].to(device)
      mask2 = data['mask2'].to(device)
      token_type_ids1 = data["token_type_ids1"].to(device)
      token_type_ids2 = data["token_type_ids2"].to(device)
      targets = data['target'].to(device)

      print(f"Iteración: {i} de {num_iteraciones}")

      outputs = model(ids1, mask1, token_type_ids1, ids2, mask2, token_type_ids2)
      fin_targets.extend(targets.cpu().detach().numpy().tolist())
      fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
  return fin_outputs, fin_targets

**Nota:** Esta celda no se encuentra ejecutada porque solo se usa cuando la evaluación de un modelo se ha hecho en una sesión distinta a su entrenamiento. Si se hace en la misma sesión, no tenemos necesidad de cargar el modelo (que acabamos de entrenar). Por el contrario, si lo entrenamos "hoy" y lo evaluamos "mañana", en vez de reentrenarlo para poder evaluarlo, solo es necesario cargarlo.

In [ ]:
if os.path.exists("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_siamese_bert_concat_lr7_BCE_50.pth"):
  model = BERTSiameseConcat()
  if device == "cuda":
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_siamese_bert_concat_lr7_BCE_50.pth"))
    print("Modelo cargado correctamente")
  else:
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_siamese_bert_concat_lr7_BCE_50.pth", map_location="cpu"))
    print("Modelo cargado correctamente con CPU")

  model.eval()
  model.to(device)

In [ ]:
for epoch in range(EPOCHS):
  outputs, targets = validationBERTSiameseConcat(epoch)
  outputs = np.array(outputs) >= 0.5
  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
  print(f"Accuracy Score = {accuracy}")
  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

Iteración: 0 de 411
Iteración: 1 de 411
Iteración: 2 de 411
Iteración: 3 de 411
Iteración: 4 de 411
Iteración: 5 de 411
Iteración: 6 de 411
Iteración: 7 de 411
Iteración: 8 de 411
Iteración: 9 de 411
Iteración: 10 de 411
Iteración: 11 de 411
Iteración: 12 de 411
Iteración: 13 de 411
Iteración: 14 de 411
Iteración: 15 de 411
Iteración: 16 de 411
Iteración: 17 de 411
Iteración: 18 de 411
Iteración: 19 de 411
Iteración: 20 de 411
Iteración: 21 de 411
Iteración: 22 de 411
Iteración: 23 de 411
Iteración: 24 de 411
Iteración: 25 de 411
Iteración: 26 de 411
Iteración: 27 de 411
Iteración: 28 de 411
Iteración: 29 de 411
Iteración: 30 de 411
Iteración: 31 de 411
Iteración: 32 de 411
Iteración: 33 de 411
Iteración: 34 de 411
Iteración: 35 de 411
Iteración: 36 de 411
Iteración: 37 de 411
Iteración: 38 de 411
Iteración: 39 de 411
Iteración: 40 de 411
Iteración: 41 de 411
Iteración: 42 de 411
Iteración: 43 de 411
Iteración: 44 de 411
Iteración: 45 de 411
Iteración: 46 de 411
Iteración: 47 de 411
It

#3. Conclusiones

## 3.1 Validación final del mejor modelo

Una vez entrenados todos los modelos y probados con el conjunto de datos de train, se procede a evaluar el modelo que mejor resultados ha conseguido con los datos de validación, datos que ***bajo ninguna circunstancia*** el modelo ha visto en ninguna otra ocasión. El modelo que mejores resultados ha obtenido ha sido un modelo con arquitectura (...)

## 3.2 Ejemplo de uso del mejor modelo

Una vez se tiene el mejor modelo entrenado y evaludado, se quiere ver la eficacia del mismo y "jugar un poco con él".

###3.2.1 Imports Generales

In [ ]:
import numpy as np
import pandas as pd
import random
import os
import warnings

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
!pip install transformers

###3.2.2 Cargar el mejor modelo

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

if os.path.exists("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_30.pth"):
  model = BERTClass()
  if device == "cuda":
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_30.pth"))
  else:
    model.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/ProyectoIA_investigacion/modelo_single_bert_30.pth", map_location="cpu"))

  model.eval()
  model.to(device)

###3.2.3 Pruebas

In [ ]:
texto_ejemplo = "Bad potatoes."

In [ ]:
def codificador(texto, tokenizador):
  encoder_input = tokenizador(
            texto,
            None,
            add_special_tokens=True,
            truncation=True,
            max_length=512,
            pad_to_max_length=True,
            return_tensors = "pt",
            return_token_type_ids=True
            )
  return {
    'ids': encoder_input["input_ids"],
    'mask': encoder_input["attention_mask"],
    "token_type_ids": encoder_input["token_type_ids"]
  }

In [ ]:
inputs = codificador(texto_ejemplo, tokenizerB)

In [ ]:
# Ensure that the model is also on the same device
model = model.to(device)

with torch.no_grad():
    output = model.forward(**inputs)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
inputs = tokenizerB(texto_ejemplo, padding=True, truncation=True, return_tensors='pt')

inputs


{'input_ids': tensor([[ 101, 2919, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [ ]:
ids = inputs["input_ids"]
token = inputs["token_type_ids"]
mask = inputs["attention_mask"]

In [ ]:
# Move tensors to the same device as the model
ids = ids.to(device)
token = token.to(device)
mask = mask.to(device)

# Ensure that the model is also on the same device
model = model.to(device)

with torch.no_grad():
    output = model.forward(ids, token, mask)

In [ ]:
output

tensor([[0.5134]], device='cuda:0')

- https://huggingface.co/mrcaelumn/yelp_restaurant_review_sentiment_analysis?text=I+like+you.+I+love+you (distil bert, ya para usar)
- https://huggingface.co/lxyuan/distilbert-base-multilingual-cased-sentiments-student (ya para usar)

## 3.3 Consideraciones finales